# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
//BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D_FirstPeriodStudy");

Project name is set to 'OscillatingDroplet3D_FirstPeriodStudy'.
Opening existing database 'D:\local\OscillatingDroplet3D_FirstPeriodStudy'.


In [ ]:
//add database manually:
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_ConvergenceStudy");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_ConvergenceStudy'.


## Observing sessions

In [ ]:
var workSess = databases.Pick(0).Sessions;
workSess

#0: OscillatingDroplet3D_ConvergenceStudy	OD3D_J1458k3_wallBC_amr1_m3_Oh01_eta04*	06/29/2022 21:45:43	08ba3084...
#1: OscillatingDroplet3D_ConvergenceStudy	OD3D_J1458k3_wallBC_amr1_m4_Oh01_eta04*	06/29/2022 21:45:43	696f13fa...
#2: OscillatingDroplet3D_ConvergenceStudy	OD3D_J250k3_wallBC_amr1_m4_Oh01_eta04_dtmin*	06/29/2022 21:45:47	3754ac89...
#3: OscillatingDroplet3D_ConvergenceStudy	OD3D_J1458k3_wallBC_amr1_m2_Oh01_eta07*	06/29/2022 21:45:43	71ac6bd9...
#4: OscillatingDroplet3D_ConvergenceStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	06/29/2022 21:45:44	f3e1c83d...
#5: OscillatingDroplet3D_ConvergenceStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_dtmin*	06/29/2022 21:45:44	f35358b8...
#6: OscillatingDroplet3D_ConvergenceStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_dtmin*	06/29/2022 21:45:43	5b13f288...
#7: OscillatingDroplet3D_ConvergenceStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_dtmin*	06/29/2022 21:45:43	cd275b94...
#8: OscillatingDroplet3D_ConvergenceStudy	OD3D_J250k3_wallBC_amr1_m3_

In [ ]:
//workSess.Pick(22).Delete(true);

Session d04780e2-17f7-4629-b57e-44b271caea15 deleted.


In [ ]:
// var delSess = workSess.Skip(0).Take(11);
// delSess

In [ ]:
// foreach(var dS in delSess) {
//     dS.Delete(true);
// }

In [ ]:
var sess = workSess.Pick(14);
sess

OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07	04/08/2022 11:31:37	de146e9e...

In [ ]:
sess.GetSessionDirectory()

\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy\sessions\de146e9e-87ad-4caf-b928-7ba443a0d324

In [ ]:
sess.DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Feb21_132417

In [ ]:
//sess.Timesteps

In [ ]:
sess.Timesteps.Skip(10).Every(10).Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D_FirstPeriodStudy__OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart1__e50468c2-046b-4232-83e4-cd214ccecfc1


In [ ]:
using System.IO;

# Study Overview

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta04", "m2_Oh01_eta07", "m3_Oh01_eta04", "m4_Oh01_eta04", "m4_Oh01_eta07"};
string[] gridS = new string[] { "J250", "J686", "J1458" };
bool[] dtmin = new bool[] { false, true };
bool[] ShouldAnaInit = new bool[] { false };
bool[] useNewton = new bool[] { false };

In [ ]:
var studySess = new List<ISessionInfo>();
foreach(string _case in caseS) {
    foreach(string _grid in gridS) {
        foreach(bool _min in dtmin) {
            foreach(var sess in workSess) {
                if(sess.Name.Contains(_case) && sess.Name.Contains(_grid)) {
                    if(!(_min ^ sess.Name.Contains("_dtmin"))) {
                        studySess.Add(sess);
                        break;
                    }
                }
            }
        }
    }
}

In [ ]:
//var studySess = workSess.Take(12);
studySess

#0: OscillatingDroplet3D_ConvergenceStudy	OD3D_J250k3_wallBC_amr1_m2_Oh01_eta04	06/24/2022 15:38:30	0fe4f3b4...
#1: OscillatingDroplet3D_ConvergenceStudy	OD3D_J250k3_wallBC_amr1_m2_Oh01_eta04_dtmin*	06/29/2022 21:45:40	2f3c5c82...
#2: OscillatingDroplet3D_ConvergenceStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_dtmin*	06/29/2022 21:45:43	cd275b94...
#3: OscillatingDroplet3D_ConvergenceStudy	OD3D_J1458k3_wallBC_amr1_m2_Oh01_eta04*	06/29/2022 21:40:09	0bd4bd0b...
#4: OscillatingDroplet3D_ConvergenceStudy	OD3D_J250k3_wallBC_amr1_m2_Oh01_eta07	06/24/2022 16:23:01	12e21818...
#5: OscillatingDroplet3D_ConvergenceStudy	OD3D_J250k3_wallBC_amr1_m2_Oh01_eta07_dtmin*	06/29/2022 21:40:11	34ec436d...
#6: OscillatingDroplet3D_ConvergenceStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_dtmin*	06/24/2022 15:49:11	d2de084e...
#7: OscillatingDroplet3D_ConvergenceStudy	OD3D_J1458k3_wallBC_amr1_m2_Oh01_eta07*	06/29/2022 21:45:43	71ac6bd9...
#8: OscillatingDroplet3D_ConvergenceStudy	OD3D_J250k3_wallBC_amr1_m3_Oh0

In [ ]:
//studySess.Pick(1).Timesteps

# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
//string[] metricKeys = new string[] {"theta0", "theta90x", "theta90y", "volume"};

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[caseS.Length,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {   
for(int iMin = 0; iMin < dtmin.Length; iMin++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {

    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_dtmin") == dtmin[iMin])
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));


    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);


    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }
    Console.WriteLine("number of processed Sessions for {0}: {1}", _case, procSIs.Count);
    

    var plot = new Plot2Ddata();

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    if (dtmin[iMin]) {
        fmt.LineColor = LineColors.Red;
    } else {
        fmt.LineColor = LineColors.Blue;
    }

    //string[] gridS = new string[] { "J54", "J250", "J686", "J1458" };
    if(_grd == "J54")
        fmt.DashType = DashTypes.Dashed; 
    if(_grd == "J250")
        fmt.DashType = DashTypes.Dotted; 
    if(_grd == "J686")
        fmt.DashType = DashTypes.DotDashed; 
    if(_grd == "J1458")
        fmt.DashType = DashTypes.Solid; 
 
    string name = _case+"_BoSSS-"+_grd;
    if(dtmin[iMin])
        name = name + "-dtmin"; 
    // if(ShouldAnaInit[iAna])
    //     name = name + "-3OrdInit"; 
    // if(useNewton[iNew])
    //     name = name + "-Newton";



    double[] time = new double[0];
    double[] arLW = new double[0];
    
    foreach(var pSI in procSIs) {

        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + pSI);
            
            // var dummyPlot = new Plot2Ddata();
            // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            // PlotTableDM[iRow,iCol] = dummyPlot;
            continue;
        }

        time = time.Concat(tab["time"]).ToArray();

        double[] L = tab["L"].ToArray(); 
        double[] Wx = tab["Wx"].ToArray(); 
        double[] Wy = tab["Wy"].ToArray();
        double[] LW = new double[L.Length];
        for (int i = 0; i < L.Length; i++) {
            LW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
        }
        arLW = arLW.Concat(LW).ToArray();

    }
    
    List<double> arLW_lst = new List<double>();
    List<double> time_lst = new List<double>();
    double dataThreshold = 0.1;
    for(int i = 0; i < time.Length; i++) {
        if(arLW[i] > dataThreshold) {
            arLW_lst.Add(arLW[i]);
            time_lst.Add(time[i]);
        }
    }
    arLW = arLW_lst.ToArray();
    time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0) {
       PlotTableDM[iRow,iCol].Title = caseS[iRow];
       PlotTableDM[iRow,iCol].Xlabel = "time t";
       PlotTableDM[iRow,iCol].Ylabel = "aspect ratio L/W";
    }
}
}
}
}
}
}

OD3D_J250k3_wallBC_amr1_m2_Oh01_eta04
number of processed Sessions for m2_Oh01_eta04: 1
OD3D_J250k3_wallBC_amr1_m2_Oh01_eta04_dtmin
number of processed Sessions for m2_Oh01_eta04: 1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_dtmin
number of processed Sessions for m2_Oh01_eta04: 1
OD3D_J1458k3_wallBC_amr1_m2_Oh01_eta04
number of processed Sessions for m2_Oh01_eta04: 1
OD3D_J250k3_wallBC_amr1_m2_Oh01_eta07
number of processed Sessions for m2_Oh01_eta07: 1
OD3D_J250k3_wallBC_amr1_m2_Oh01_eta07_dtmin
number of processed Sessions for m2_Oh01_eta07: 1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_dtmin
number of processed Sessions for m2_Oh01_eta07: 1
OD3D_J1458k3_wallBC_amr1_m2_Oh01_eta07
number of processed Sessions for m2_Oh01_eta07: 1
OD3D_J250k3_wallBC_amr1_m3_Oh01_eta04
number of processed Sessions for m3_Oh01_eta04: 1
OD3D_J250k3_wallBC_amr1_m3_Oh01_eta04_dtmin
number of processed Sessions for m3_Oh01_eta04: 1
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
number of processed Sessions for m3_Oh01_eta04: 1


In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta04 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta07 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta04 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta04 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta07

## m=2 cases (eta = {0.4, 0.7})

In [ ]:
var pltTab1 = PlotTableDM[0,0];

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 2.5;
var gp = pltTab1.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta04 
 
 
 m2Oh01eta04BoSSS-J250 
 
 
 
 
 m2 O h01 e ta04 B oSSS-J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M388.7,93.1 L442.1,93.1 M71.9,79.2 L73.9,79.1 L76.0,79.4 L78.0,79.9 L80.0,80.6 L82.1,81.6
 L84.1,82.7 L86.1,83.9 L88.2,85.3 L90.2,86.8 L92.2,88.3 L94.3,90.0 L96.3,91.7 L98.3,93.5
 L100.4,95.3 L102.4,97.2 L104.5,99.2 L106.5,101.1 L108.5,103.2 L110.6,105.3 L112.6,107.4 L114.6,109.5
 L116.7,111.7 L118.7,114.0 L120.7,116.2 L122.8,118.5 L124.8,120.8 L126.8,123.1 L128.9,125.5 L130.9,127.9
 L132.9,130.3 L135.0,132.7 L137.0,135.1 L139.0,137.5 L141.1,140.0 L143.1,142.5 L145.1,144.9 L147.2,148.2
 L149.2,150.4 L151.2,153.2 L153.3,155.7 L155.3,157.9 L157.4,160.2 L159.4,162.6 L161.4,165.9 L163.5,168.1
 L165.5,170.5 L167.5,173.0 L169.6,175.6 L171.6,178.2 L173.6,180.8 L175.7,183.5 L177.7,186.3 L179.7,189.0
 L181.8,191.9 L183.8,195.7 L185.8,198.1 L187.9,201.8 L189.9,204.4 L191.9,207.1 L194.0,210.0 L196.0,213.0
 L198.0,216.0 L200.1,219.1 L202.1,223.3 L204.1,226.4 L206.2,229.5 L208.2,232.7 L210.3,236.0 L212.3,239.4
 L214.3,242.9 L216.4,246.4 L218.4,250.0 L220.4,253.7 L222.5,257.8 L224.5,261.3 L226.5,264.9 L228.6,268.6
 L230.6,272.5 L232.6,276.6 L234.7,280.5 L236.7,284.3 L238.7,288.1 L240.8,292.0 L242.8,296.0 L244.8,299.9
 L246.9,303.8 L248.9,307.9 L250.9,311.9 L253.0,315.9 L255.0,320.0 L257.0,324.0 L259.1,328.1 L261.1,332.1
 L263.2,336.1 L265.2,340.1 L267.2,344.1 L269.3,348.0 L271.3,351.9 L273.3,355.8 L275.4,359.7 L277.4,363.6
 L279.4,367.4 L281.5,371.2 L283.5,374.9 L285.5,378.7 L287.6,382.1 L289.6,385.9 L291.6,389.4 L293.7,393.0
 L295.7,396.6 L297.7,400.1 L299.8,403.5 L301.8,406.8 L303.8,410.1 L305.9,413.3 L307.9,416.4 L309.9,419.5
 L312.0,422.5 L314.0,425.2 L316.1,428.2 L318.1,431.1 L320.1,434.0 L322.2,436.8 L324.2,439.6 L326.2,442.1
 L328.3,444.4 L330.3,446.8 L332.3,449.2 L334.4,451.5 L336.4,453.7 L338.4,455.9 L340.5,458.0 L342.5,460.1
 L344.5,462.1 L346.6,464.1 L348.6,466.0 L350.6,467.8 L352.7,469.6 L354.7,471.0 L356.7,472.8 L358.8,474.4
 L360.8,476.0 L362.8,477.5 L364.9,478.9 L366.9,480.2 L369.0,481.5 L371.0,482.7 L373.0,483.8 L375.1,484.9
 L377.1,485.9 L379.1,486.8 L381.2,487.7 L383.2,488.5 L385.2,489.3 L387.3,490.1 L389.3,490.7 L391.3,491.3
 L393.4,491.9 L395.4,492.4 L397.4,492.9 L399.5,493.3 L401.5,493.7 L403.5,494.0 L405.6,494.2 L407.6,494.4
 L409.6,494.6 L411.7,494.7 L413.7,494.8 L415.7,494.8 L417.8,494.8 L419.8,494.7 L421.9,494.6 L423.9,494.4
 L425.9,494.2 L428.0,494.0 L430.0,493.7 L432.0,493.3 L434.1,492.9 L436.1,492.5 L438.1,492.0 L440.2,491.5
 L442.2,491.0 L444.2,490.4 L446.3,489.7 L448.3,489.0 L450.3,488.3 L452.4,487.6 L454.4,486.8 L456.4,485.9
 L458.5,485.0 L460.5,484.1 L462.5,483.1 L464.6,482.1 L466.6,481.1 L468.6,480.0 L470.7,478.9 L472.7,477.7
 L474.8,476.5 L476.8,475.3 L478.8,474.0 L480.9,472.8 L482.9,471.4 L484.9,469.4 L487.0,468.2 L489.0,466.9
 L491.0,465.6 L493.1,464.2 L495.1,462.7 L497.1,461.2 L499.2,459.6 L501.2,458.0 L503.2,456.3 L505.3,454.6
 L507.3,452.8 L509.3,451.0 L511.4,449.2 L513.4,447.4 L515.4,445.6 L517.5,443.8 L519.5,441.0 L521.5,439.2
 L523.6,437.4 L525.6,435.6 L527.7,433.9 L529.7,432.0 L531.7,430.2 L533.8,428.1 L535.8,426.2 L537.8,424.2
 L539.9,422.1 L541.9,419.1 L543.9,417.2 L546.0,415.3 L548.0,413.3 L550.0,411.3 L552.1,409.4 L554.1,407.4
 L556.1,405.3 L558.2,403.3 L560.2,401.2 L562.2,399.1 L564.3,397.0 L566.3,394.9 L568.3,392.8 L570.4,390

In [ ]:
// for(int i = 0; i < 4; i++) {
//     Plot2Ddata dat = new Plot2Ddata();
//     var grp = PlotTableDM[i,0].dataGroups[0];
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

In [ ]:
var pltTab2 = PlotTableDM[1,0];

In [ ]:
pltTab2.XrangeMin = 0;
pltTab2.XrangeMax = 2.5;
var gp = pltTab2.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta07 
 
 
 m2Oh01eta07BoSSS-J250 
 
 
 
 
 m2 O h01 e ta07 B oSSS-J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M388.7,93.1 L442.1,93.1 M71.9,89.5 L73.9,88.4 L76.0,90.4 L78.0,93.0 L80.0,96.0 L82.1,99.3
 L84.1,102.9 L86.1,106.5 L88.2,110.1 L90.2,113.6 L92.2,117.0 L94.3,120.2 L96.3,123.4 L98.3,126.5
 L100.4,129.7 L102.4,132.8 L104.5,136.0 L106.5,139.9 L108.5,144.3 L110.6,145.3 L112.6,149.3 L114.6,153.5
 L116.7,156.3 L118.7,159.6 L120.7,162.6 L122.8,165.4 L124.8,168.4 L126.8,171.5 L128.9,175.0 L130.9,177.6
 L132.9,180.8 L135.0,183.1 L137.0,186.8 L139.0,190.1 L141.1,193.1 L143.1,195.8 L145.1,198.4 L147.2,201.4
 L149.2,203.9 L151.2,203.9 L153.3,207.3 L155.3,210.6 L157.4,213.5 L159.4,216.3 L161.4,219.0 L163.5,221.6
 L165.5,224.0 L167.5,226.4 L169.6,228.7 L171.6,230.9 L173.6,233.1 L175.7,235.2 L177.7,237.4 L179.7,239.5
 L181.8,241.7 L183.8,242.8 L185.8,245.5 L187.9,247.9 L189.9,250.2 L191.9,252.3 L194.0,254.4 L196.0,256.4
 L198.0,258.3 L200.1,260.3 L202.1,262.2 L204.1,264.2 L206.2,266.1 L208.2,268.0 L210.3,270.3 L212.3,272.3
 L214.3,274.3 L216.4,275.3 L218.4,278.2 L220.4,279.9 L222.5,282.1 L224.5,284.4 L226.5,287.0 L228.6,289.4
 L230.6,291.6 L232.6,294.2 L234.7,296.3 L236.7,298.5 L238.7,300.8 L240.8,303.0 L242.8,305.3 L244.8,307.7
 L246.9,310.0 L248.9,312.4 L250.9,314.9 L253.0,315.6 L255.0,318.9 L257.0,323.1 L259.1,325.5 L261.1,328.0
 L263.2,330.4 L265.2,333.0 L267.2,336.4 L269.3,339.3 L271.3,341.5 L273.3,344.2 L275.4,346.9 L277.4,348.4
 L279.4,351.8 L281.5,353.9 L283.5,357.2 L285.5,360.1 L287.6,363.0 L289.6,366.0 L291.6,368.9 L293.7,371.8
 L295.7,376.6 L297.7,379.4 L299.8,382.2 L301.8,384.5 L303.8,387.3 L305.9,389.9 L307.9,392.4 L309.9,395.5
 L312.0,398.6 L314.0,401.1 L316.1,404.1 L318.1,406.8 L320.1,409.5 L322.2,413.3 L324.2,416.4 L326.2,419.1
 L328.3,421.8 L330.3,424.6 L332.3,427.7 L334.4,429.7 L336.4,432.9 L338.4,435.7 L340.5,438.4 L342.5,441.1
 L344.5,443.7 L346.6,445.8 L348.6,448.6 L350.6,452.8 L352.7,455.2 L354.7,457.5 L356.7,459.7 L358.8,461.9
 L360.8,464.1 L362.8,466.0 L364.9,468.3 L366.9,470.5 L369.0,472.7 L371.0,475.0 L373.0,476.6 L375.1,478.7
 L377.1,480.5 L379.1,482.3 L381.2,484.1 L383.2,486.0 L385.2,487.3 L387.3,489.0 L389.3,490.6 L391.3,492.5
 L393.4,494.1 L395.4,495.6 L397.4,497.0 L399.5,498.4 L401.5,499.6 L403.5,500.8 L405.6,502.3 L407.6,503.7
 L409.6,504.6 L411.7,505.6 L413.7,506.8 L415.7,507.7 L417.8,508.6 L419.8,509.4 L421.9,510.1 L423.9,510.9
 L425.9,511.6 L428.0,512.3 L430.0,512.9 L432.0,513.6 L434.1,514.1 L436.1,514.8 L438.1,515.4 L440.2,516.0
 L442.2,516.2 L444.2,516.7 L446.3,517.1 L448.3,517.4 L450.3,517.7 L452.4,518.0 L454.4,518.3 L456.4,518.4
 L458.5,518.7 L460.5,518.9 L462.5,519.0 L464.6,519.0 L466.6,519.0 L468.6,519.1 L470.7,519.1 L472.7,519.1
 L474.8,519.0 L476.8,518.9 L478.8,518.7 L480.9,518.4 L482.9,518.3 L484.9,517.9 L487.0,517.7 L489.0,517.4
 L491.0,517.1 L493.1,516.8 L495.1,516.5 L497.1,516.1 L499.2,515.7 L501.2,515.2 L503.2,514.7 L505.3,514.2
 L507.3,513.7 L509.3,513.1 L511.4,512.5 L513.4,511.9 L515.4,511.4 L517.5,510.8 L519.5,510.1 L521.5,509.2
 L523.6,508.7 L525.6,508.0 L527.7,507.3 L529.7,506.2 L531.7,505.2 L533.8,504.4 L535.8,503.5 L537.8,502.6
 L539.9,501.5 L541.9,500.6 L543.9,499.5 L546.0,498.6 L548.0,497.6 L550.0,496.6 L552.1,495.6 L554.1,494.6
 L556.1,493.5 L558.2,492.4 L560.2,491.3 L562.2,490.2 L564.3,489.0 L566.3,487.4 L568.3,486.7 L570.4,485.4
 L572.4,484.4 

In [ ]:
// for(int i = 0; i < 4; i++) {
//     Plot2Ddata dat = new Plot2Ddata();
//     var grp = PlotTableDM[i,0].dataGroups[0];
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## m=3 cases (eta = 0.4)

In [ ]:
var pltTab3 = PlotTableDM[2,0];

In [ ]:
pltTab3.XrangeMin = 0;
pltTab3.XrangeMax = 0.5;
var gp = pltTab3.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.98 
 
 
 
 
 1 
 
 
 
 
 1.02 
 
 
 
 
 1.04 
 
 
 
 
 1.06 
 
 
 
 
 1.08 
 
 
 
 
 1.1 
 
 
 
 
 1.12 
 
 
 
 
 1.14 
 
 
 
 
 1.16 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m3 O h01 e ta04 
 
 
 m3Oh01eta04BoSSS-J250 
 
 
 
 
 m3 O h01 e ta04 B oSSS-J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M388.7,93.1 L442.1,93.1 M80.2,463.7 L86.9,463.7 L93.6,463.0 L100.3,461.4 L107.0,458.1 L113.7,453.1
 L120.4,446.8 L127.1,439.3 L133.8,432.4 L140.5,421.8 L147.2,413.7 L153.9,403.8 L160.6,390.4 L167.3,378.2
 L174.0,367.1 L180.7,356.7 L187.4,346.7 L194.1,337.0 L200.8,327.5 L207.5,318.3 L214.2,309.2 L220.9,300.4
 L227.6,341.2 L234.3,330.3 L241.0,323.8 L247.7,317.6 L254.4,309.2 L261.1,301.0 L267.8,295.4 L274.5,288.1
 L281.2,281.7 L287.9,275.3 L294.6,268.9 L301.3,262.6 L308.0,256.5 L314.7,250.5 L321.4,237.8 L328.1,232.9
 L334.8,222.9 L341.5,218.3 L348.2,230.8 L354.9,224.2 L361.6,211.8 L368.3,206.4 L375.0,200.0 L381.7,183.1
 L388.4,170.1 L395.1,165.4 L401.8,173.1 L408.5,170.2 L415.2,165.9 L421.8,161.9 L428.5,157.6 L435.2,150.8
 L441.9,146.6 L448.6,143.0 L455.3,139.8 L462.0,135.9 L468.7,132.2 L475.4,129.5 L482.1,125.9 L488.8,128.5
 L495.5,123.7 L502.2,122.0 L508.9,120.5 L515.6,122.9 L522.3,118.0 L529.0,117.5 L535.7,117.3 L542.4,118.3
 L549.1,124.8 L555.8,125.6 L562.5,123.3 L569.2,123.7 L575.9,124.5 L582.6,125.7 L589.3,127.2 L596.0,129.0
 L602.7,131.0 L609.4,133.3 L616.1,136.8 L622.8,142.3 L629.5,139.3 L636.2,151.2 L642.9,152.5 L649.6,155.4
 L656.3,158.4 L663.0,161.4 L669.7,164.5 L676.4,161.9 L683.1,164.7 L689.8,167.7 L696.5,170.9 L703.2,174.4
 L709.9,178.1 L716.6,184.4 L723.3,188.3 L730.0,194.1 L736.7,197.8 L743.4,201.7 L750.1,205.4 '/> 
 
 m3Oh01eta04BoSSS-J250-dtmin 
 
 
 m3 O h01 e ta04 B oSSS-J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M388.7,117.1 L442.1,117.1 M80.2,463.7 L82.2,463.7 L84.2,464.6 L86.2,465.7 L88.2,466.4 L90.2,466.5
 L92.3,466.4 L94.3,465.9 L96.3,465.2 L98.3,464.3 L100.3,463.2 L102.3,461.9 L104.3,460.5 L106.3,458.8
 L108.3,457.1 L110.3,455.1 L112.4,453.0 L114.4,450.8 L116.4,448.4 L118.4,445.9 L120.4,443.0 L122.4,442.6
 L124.4,438.9 L126.4,435.8 L128.4,432.9 L130.4,430.1 L132.5,427.3 L134.5,424.5 L136.5,421.6 L138.5,418.5
 L140.5,415.2 L142.5,416.3 L144.5,413.5 L146.5,408.8 L148.5,404.7 L150.5,400.8 L152.5,397.2 L154.6,397.0
 L156.6,394.0 L158.6,391.0 L160.6,388.0 L162.6,385.0 L164.6,382.1 L166.6,379.1 L168.6,376.1 L170.6,373.2
 L172.6,370.2 L174.7,362.6 L176.7,359.7 L178.7,356.9 L180.7,354.0 L182.7,351.2 L184.7,348.4 L186.7,345.6
 L188.7,342.9 L190.7,340.1 L192.7,337.4 L194.8,334.6 L196.8,331.9 L198.8,329.2 L200.8,326.5 L202.8,323.8
 L204.8,321.2 L206.8,318.5 L208.8,315.9 L210.8,313.2 L212.8,310.6 L214.8,308.0 L216.9,305.4 L218.9,302.8
 L220.9,348.9 L222.9,346.6 L224.9,344.3 L226.9,342.0 L228.9,339.7 L230.9,334.1 L232.9,332.2 L234.9,330.3
 L237.0,328.4 L239.0,326.4 L241.0,324.5 L243.0,322.5 L245.0,320.4 L247.0,316.8 L249.0,314.6 L251.0,312.6
 L253.0,310.7 L255.0,307.3 L257.1,306.1 L259.1,302.9 L261.1,301.2 L263.1,299.6 L265.1,297.9 L267.1,296.3
 L269.1,294.6 L271.1,292.8 L273.1,291.0 L275.1,289.2 L277.2,287.4 L279.2,285.5 L281.2,283.7 L283.2,281.8
 L285.2,279.9 L287.2,278.0 L289.2,276.1 L291.2,274.2 L293.2,272.3 L295.2,270.4 L297.2,268.6 L299.3,266.7
 L301.3,264.9 L303.3,263.0 L305.3,261.2 L307.3,259.4 L309.3,257.6 L311.3,255.8 L313.3,248.1 L315.3,246.9
 L317.3,245.1 L319.4,244.2 L321.4,243.0 L323.4,241.7 L325.4,240.3 L327.4,230.4 L

In [ ]:
// foreach(var grp in pltTab1.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + pltTab1.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## m=4 cases (eta = {0.4, 0.7})

In [ ]:
var pltTab4 = PlotTableDM[3,0];

In [ ]:
pltTab4.XrangeMin = 0;
pltTab4.XrangeMax = 0.5;
var gp = pltTab4.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 1.25 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h01 e ta04 
 
 
 m4Oh01eta04BoSSS-J250 
 
 
 
 
 m4 O h01 e ta04 B oSSS-J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M388.7,93.1 L442.1,93.1 M80.2,117.9 L86.9,118.3 L93.6,117.9 L100.3,117.5 L107.0,117.2 L113.7,117.1
 L120.4,117.0 L127.1,116.9 L133.8,116.8 L140.5,116.6 L147.2,116.4 L153.9,116.2 L160.6,117.0 L167.3,116.2
 L174.0,115.4 L180.7,114.8 L187.4,114.3 L194.1,113.8 L200.8,113.3 L207.5,112.9 L214.2,112.4 L220.9,111.7
 L227.6,111.0 L234.3,110.1 L241.0,109.2 L247.7,108.2 L254.4,107.1 L261.1,105.9 L267.8,104.7 L274.5,105.6
 L281.2,104.6 L287.9,102.4 L294.6,102.7 L301.3,100.7 L308.0,99.3 L314.7,98.3 L321.4,97.7 L328.1,97.3
 L334.8,97.2 L341.5,97.3 L348.2,99.2 L354.9,99.6 L361.6,99.4 L368.3,101.3 L375.0,101.7 L381.7,102.8
 L388.4,104.8 L395.1,107.0 L401.8,109.2 L408.5,111.2 L415.2,115.1 L421.8,121.7 L428.5,130.9 L435.2,135.6
 L441.9,139.7 L448.6,144.4 L455.3,150.1 L462.0,157.0 L468.7,164.2 L475.4,171.8 L482.1,179.9 L488.8,189.4
 L495.5,198.2 L502.2,212.3 L508.9,224.2 L515.6,233.1 L522.3,242.4 L529.0,251.9 L535.7,270.6 L542.4,281.5
 L549.1,289.5 L555.8,301.9 L562.5,322.1 L569.2,334.7 L575.9,343.0 L582.6,358.8 L589.3,370.0 L596.0,379.6
 L602.7,388.4 L609.4,396.8 L616.1,404.5 L622.8,411.7 L629.5,418.3 L636.2,424.3 L642.9,429.6 L649.6,434.2
 L656.3,438.0 L663.0,441.1 L669.7,443.2 L676.4,444.5 L683.1,444.6 L689.8,446.0 L696.5,445.4 L703.2,443.9
 L709.9,441.7 L716.6,437.8 L723.3,437.9 L730.0,434.6 L736.7,430.8 L743.4,426.6 L750.1,422.0 '/> 
 
 m4Oh01eta04BoSSS-J250-dtmin 
 
 
 m4 O h01 e ta04 B oSSS-J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M388.7,117.1 L442.1,117.1 M80.2,117.9 L82.2,118.3 L84.2,117.9 L86.2,117.5 L88.2,117.3 L90.2,117.1
 L92.3,117.0 L94.3,117.0 L96.3,117.0 L98.3,117.1 L100.3,117.1 L102.3,117.1 L104.3,117.1 L106.3,117.2
 L108.3,117.2 L110.3,117.2 L112.4,117.2 L114.4,117.2 L116.4,117.2 L118.4,117.2 L120.4,117.1 L122.4,117.1
 L124.4,117.1 L126.4,117.0 L128.4,116.9 L130.4,116.9 L132.5,116.8 L134.5,116.7 L136.5,116.6 L138.5,116.6
 L140.5,116.5 L142.5,116.4 L144.5,116.3 L146.5,116.2 L148.5,116.0 L150.5,115.9 L152.5,115.8 L154.6,115.7
 L156.6,115.6 L158.6,115.9 L160.6,115.3 L162.6,114.8 L164.6,114.5 L166.6,114.3 L168.6,114.2 L170.6,114.1
 L172.6,114.1 L174.7,114.0 L176.7,114.0 L178.7,114.0 L180.7,113.9 L182.7,113.8 L184.7,113.8 L186.7,113.7
 L188.7,113.6 L190.7,113.5 L192.7,113.4 L194.8,113.3 L196.8,113.1 L198.8,113.0 L200.8,112.9 L202.8,112.7
 L204.8,112.6 L206.8,112.4 L208.8,112.2 L210.8,112.0 L212.8,111.8 L214.8,111.6 L216.9,111.4 L218.9,111.2
 L220.9,110.9 L222.9,110.6 L224.9,110.4 L226.9,110.1 L228.9,109.8 L230.9,109.5 L232.9,109.2 L234.9,108.9
 L237.0,108.6 L239.0,108.3 L241.0,107.9 L243.0,107.6 L245.0,107.3 L247.0,106.9 L249.0,106.6 L251.0,106.3
 L253.0,105.9 L255.0,105.6 L257.1,105.2 L259.1,104.9 L261.1,104.5 L263.1,106.1 L265.1,105.8 L267.1,104.5
 L269.1,104.8 L271.1,103.4 L273.1,102.4 L275.1,101.7 L277.2,103.8 L279.2,103.6 L281.2,102.2 L283.2,101.2
 L285.2,102.9 L287.2,101.5 L289.2,100.7 L291.2,100.1 L293.2,99.8 L295.2,99.5 L297.2,99.3 L299.3,99.1
 L301.3,99.0 L303.3,98.9 L305.3,98.8 L307.3,98.7 L309.3,98.6 L311.3,98.5 L313.3,98.4 L315.3,98.4
 L317.3,98.3 L319.4,98.3 L321.4,98.3 L323.4,98.3 L325.4,98.3 L327.4,98.3 L329.4,98.4 L331.4,98.4
 L333.4,98.5 L335.4,98.6 L337.4,98.7 L339.5,98.9 L341.5,99.0 L343.5,99.2 L345.5,

In [ ]:
// for(int i = 7; i < 12; i++) {
//     Plot2Ddata dat = new Plot2Ddata();
//     var grp = PlotTableDM[i,0].dataGroups[0];
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m4/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

In [ ]:
var pltTab5 = PlotTableDM[4,0];

In [ ]:
pltTab5.XrangeMin = 0;
pltTab5.XrangeMax = 0.2;
var gp = pltTab5.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.36 
 
 
 
 
 1.37 
 
 
 
 
 1.38 
 
 
 
 
 1.39 
 
 
 
 
 1.4 
 
 
 
 
 1.41 
 
 
 
 
 1.42 
 
 
 
 
 1.43 
 
 
 
 
 1.44 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h01 e ta07 
 
 
 m4Oh01eta07BoSSS-J250 
 
 
 
 
 m4 O h01 e ta07 B oSSS-J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M388.7,93.1 L442.1,93.1 M80.2,526.4 L96.9,528.4 L113.7,525.3 L130.4,519.6 L147.2,514.9 L163.9,509.6
 L180.7,505.4 L197.4,500.7 L214.2,495.0 L230.9,490.6 L247.7,497.8 L264.4,490.5 L281.2,481.9 L297.9,479.4
 L314.7,471.0 L331.4,462.4 L348.2,452.1 L364.9,440.8 L381.7,430.3 L398.4,413.5 L415.2,408.8 L431.9,398.7
 L448.6,388.8 L465.4,383.8 L482.1,376.1 L498.9,365.3 L515.6,353.2 L532.4,339.3 L549.1,324.3 L565.9,309.5
 L582.6,292.9 L599.4,275.3 L616.1,260.4 L632.9,268.3 L649.6,256.9 L666.4,239.0 L683.1,194.7 L699.9,174.0
 L716.6,159.9 L733.4,138.3 L750.1,134.2 '/> 
 
 m4Oh01eta07BoSSS-J250-dtmin 
 
 
 m4 O h01 e ta07 B oSSS-J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M388.7,117.1 L442.1,117.1 M80.2,526.4 L85.2,528.4 L90.2,526.3 L95.3,523.1 L100.3,519.9 L105.3,518.0
 L110.3,516.7 L115.4,514.7 L120.4,513.9 L125.4,513.3 L130.4,512.8 L135.5,512.4 L140.5,511.2 L145.5,510.7
 L150.5,510.2 L155.6,509.6 L160.6,508.9 L165.6,508.1 L170.6,507.2 L175.7,506.3 L180.7,503.2 L185.7,502.2
 L190.7,501.1 L195.8,500.0 L200.8,498.8 L205.8,497.6 L210.8,496.3 L215.9,495.0 L220.9,493.6 L225.9,488.2
 L230.9,500.9 L236.0,497.4 L241.0,493.1 L246.0,489.0 L251.0,484.3 L256.0,481.1 L261.1,478.4 L266.1,473.3
 L271.1,471.2 L276.1,462.5 L281.2,456.7 L286.2,456.2 L291.2,461.3 L296.2,476.8 L301.3,473.1 L306.3,467.7
 L311.3,463.2 L316.3,457.1 L321.4,449.6 L326.4,446.0 L331.4,442.5 L336.4,439.0 L341.5,435.8 L346.5,435.5
 L351.5,433.9 L356.5,431.4 L361.6,429.0 L366.6,430.7 L371.6,429.3 L376.6,428.5 L381.7,426.0 L386.7,426.7
 L391.7,422.8 L396.7,421.7 L401.8,420.8 L406.8,417.7 L411.8,416.5 L416.8,412.6 L421.8,411.6 L426.9,407.0
 L431.9,405.1 L436.9,402.5 L441.9,399.8 L447.0,396.6 L452.0,393.6 L457.0,390.2 L462.0,387.1 L467.1,383.4
 L472.1,379.6 L477.1,375.7 L482.1,373.1 L487.2,369.1 L492.2,365.4 L497.2,361.3 L502.2,357.2 L507.3,353.0
 L512.3,348.7 L517.3,344.4 L522.3,340.0 L527.4,335.6 L532.4,331.1 L537.4,326.5 L542.4,325.5 L547.5,321.0
 L552.5,316.4 L557.5,321.1 L562.5,326.3 L567.6,316.9 L572.6,311.9 L577.6,308.4 L582.6,300.6 L587.6,289.9
 L592.7,283.8 L597.7,277.9 L602.7,272.0 L607.7,266.1 L612.8,260.4 L617.8,254.6 L622.8,249.8 L627.8,243.1
 L632.9,259.3 L637.9,254.3 L642.9,256.6 L647.9,243.2 L653.0,235.9 L658.0,226.7 L663.0,216.3 L668.0,210.0
 L673.1,207.2 L678.1,191.6 L683.1,194.6 L688.1,179.4 L693.2,166.9 L698.2,157.5 L703.2,167.7 L708.2,162.4
 L713.3,156.6 L718.3,149.9 L723.3,141.4 L728.3,149.4 L733.4,138.0 L738.4,125.3 L743.4,108.9 L748.4,111.8
 L750.1,109.3 '/> 
 
 m4Oh01eta07BoSSS-J686 
 
 
 m4 O h01 e ta07 B oSSS-J686 
 
 
 
 
 
 m4Oh01eta07BoSSS-J686-dtmin 
 
 
 m4 O h01 e ta07 B oSSS-J686-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='12.0,6.0,3.0,6.0' d='M388.7,165.1 L442.1,165.1 M80.2,521.5 L85.2,522.6 L90.2,521.5 L95.3,520.9 L100.3,520.5 L105.3,520.0
 L110.3,519.5 L115.4,519.0 L120.4,518.4 L125.4,517.7 L130.4,517.0 L135.5,516.2 L140.5,515.4 L145.5,514.5
 L150.5,513.6 L155.6,512.6 L160.6,511.6 L165.6,510.5 L170.6,509.3 L175.7,508.1 L180.7,506.8 L185.7,505.5
 L190.7,504.1 L195.8,507.5 L200.8,504.3 L205.8,501.9 L210.8,499.6 L215.9,497.5 L220.9,495.4 L225.9,493.4
 L230.9,496.1 L236.0,492.1 L241.0,489.0 L246

In [ ]:
// for(int i = 7; i < 12; i++) {
//     Plot2Ddata dat = new Plot2Ddata();
//     var grp = PlotTableDM[i,0].dataGroups[0];
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m4/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

# Plotting energies

In [ ]:
string[] energyKeys = new string[] {"kineticEnergy", "surfaceEnergy", "totalEnergy"}; //, "kineticDissipation"};
double surfEnrgy0 = Math.PI;    // surface energy for a sphere with r = 1 and sigma = 1 on a quarter domain

In [ ]:
Plot2Ddata[][] PlotTableEnrgy = new Plot2Ddata[caseS.Length][];
for(int iRow = 0; iRow < PlotTableEnrgy.GetLength(0); iRow++) { // loop over cases
//for(int iCol = 0; iCol < energyKeys.Length; iCol++) {
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {
for(int iMin = 0; iMin < dtmin.Length; iMin++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
        
    string _case = caseS[iRow];
    string _grd = gridS[iGrd];
       
    // if(_case == "case2" || _case == "case3")
    //     continue; 
        
    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                    && sess.Name.Contains(_grd)
                                    && (sess.Name.Contains("_dtmin") == dtmin[iMin])
                                    && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                    && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }

    if(SIs.IsNullOrEmpty()) 
    continue;
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    //fmt.LineColor = LineColors.Blue;
    if (dtmin[iMin]) {
        fmt.LineColor = LineColors.Red;
    } else {
        fmt.LineColor = LineColors.Blue;
    }

    //string[] gridS = new string[] { "J54", "J250", "J686", "J1458" };
    if(_grd == "J54")
        fmt.DashType = DashTypes.Dashed; 
    if(_grd == "J250")
        fmt.DashType = DashTypes.Dotted; 
    if(_grd == "J686")
        fmt.DashType = DashTypes.DotDashed; 
    if(_grd == "J1458")
        fmt.DashType = DashTypes.Solid; 

    
    var fmtSE = fmt.CloneAs();   
    var fmtKE = fmt.CloneAs();   
    var fmtTE = fmt.CloneAs();    

    string description = _grd;
    if(dtmin[iMin])
        description = description + "-dtmin"; 
    string nameSE = _case+" surfaceEnergy "+description;
    string nameKE = _case+" kineticEnergy "+description;
    string nameTE = _case+" totalEnergy "+description;


    List<double> time = new List<double>();
    List<double> procSurfEnrgy = new List<double>();
    List<double> kinEnrgy = new List<double>();
    List<double> totEnrgy = new List<double>();


    foreach(var pSI in procSIs) {

        //Console.WriteLine("processing Session: {0}", pSI.Name);
        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("EnergyLogValues.txt", '\t');
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + pSI);
            
            // var dummyPlot = new Plot2Ddata();
            // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            // PlotTableDM[iRow,iCol] = dummyPlot;
            continue;
        }
        
        time = time.Concat(tab["time"]).ToList();
        
        foreach(double sEnrgyVal in tab["surfaceEnergy"]) {
            procSurfEnrgy.Add(sEnrgyVal - surfEnrgy0);
        }  
        kinEnrgy = kinEnrgy.Concat(tab["kineticEnergy"]).ToList();
        totEnrgy = totEnrgy.Concat(tab["totalEnergy"]).ToList(); 

    }

    var plotSE = new Plot2Ddata();
    var plotKE = new Plot2Ddata();
    var plotTE = new Plot2Ddata();

    plotSE.AddDataGroup(nameSE, time, procSurfEnrgy, fmtSE);
    plotKE.AddDataGroup(nameKE, time, kinEnrgy, fmtKE);
    plotTE.AddDataGroup(nameTE, time, totEnrgy, fmtTE);

        
    if(PlotTableEnrgy[iRow]==null) {
        PlotTableEnrgy[iRow] = new Plot2Ddata[] {plotSE, plotKE, plotTE};
    } else {
        PlotTableEnrgy[iRow][0] = PlotTableEnrgy[iRow][0].Merge(plotSE);
        PlotTableEnrgy[iRow][1] = PlotTableEnrgy[iRow][1].Merge(plotKE);
        PlotTableEnrgy[iRow][2] = PlotTableEnrgy[iRow][2].Merge(plotTE);
    }
        
}
}
}
}
}
//}

OD3D_J250k3_wallBC_amr1_m2_Oh01_eta04
OD3D_J250k3_wallBC_amr1_m2_Oh01_eta04_dtmin
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_dtmin
OD3D_J1458k3_wallBC_amr1_m2_Oh01_eta04
OD3D_J250k3_wallBC_amr1_m2_Oh01_eta07
OD3D_J250k3_wallBC_amr1_m2_Oh01_eta07_dtmin
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_dtmin
OD3D_J1458k3_wallBC_amr1_m2_Oh01_eta07
OD3D_J250k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J250k3_wallBC_amr1_m3_Oh01_eta04_dtmin
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_dtmin
OD3D_J1458k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J250k3_wallBC_amr1_m4_Oh01_eta04
OD3D_J250k3_wallBC_amr1_m4_Oh01_eta04_dtmin
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_dtmin
OD3D_J1458k3_wallBC_amr1_m4_Oh01_eta04
OD3D_J250k3_wallBC_amr1_m4_Oh01_eta07
OD3D_J250k3_wallBC_amr1_m4_Oh01_eta07_dtmin
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07_dtmin
OD3D_J1458k3_wallBC_amr1_m4_Oh01_eta07


In [ ]:
PlotTableEnrgy

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 7 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 8 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 9 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 10 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 11 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 12 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 13 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 14 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 15 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 16 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 17 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 18 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 19 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 20 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> 

## m=2 cases (eta = {0.4, 0.7})

In [ ]:
Plot2Ddata[] pltTabE1 = PlotTableEnrgy[0];

In [ ]:
pltTabE1[0].XrangeMin = 0;
pltTabE1[0].XrangeMax = 2.5;
var gp = pltTabE1[0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.02 
 
 
 
 
 0.04 
 
 
 
 
 0.06 
 
 
 
 
 0.08 
 
 
 
 
 0.1 
 
 
 
 
 0.12 
 
 
 
 
 0.14 
 
 
 
 
 0.16 
 
 
 
 
 0.18 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta04 surfaceEnergy J250 
 
 
 
 
 m2 O h01 e ta04 surfaceEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,37.7 L64.3,37.8 L66.3,38.2 L68.4,39.0 L70.5,40.3 L72.5,41.8
 L74.6,43.7 L76.6,45.9 L78.7,48.4 L80.8,51.1 L82.8,54.1 L84.9,57.3 L87.0,60.7 L89.0,64.3
 L91.1,68.1 L93.2,72.0 L95.2,76.2 L97.3,80.4 L99.3,84.8 L101.4,89.4 L103.5,94.0 L105.5,98.8
 L107.6,103.7 L109.7,108.7 L111.7,113.9 L113.8,119.1 L115.9,124.4 L117.9,129.9 L120.0,135.4 L122.0,141.0
 L124.1,146.7 L126.2,152.5 L128.2,158.3 L130.3,164.3 L132.4,170.3 L134.4,176.3 L136.5,182.5 L138.6,188.7
 L140.6,195.1 L142.7,201.3 L144.7,207.7 L146.8,214.2 L148.9,220.7 L150.9,227.3 L153.0,233.8 L155.1,240.4
 L157.1,247.1 L159.2,253.9 L161.3,260.6 L163.3,267.4 L165.4,274.2 L167.4,281.0 L169.5,287.9 L171.6,294.8
 L173.6,301.7 L175.7,308.5 L177.8,315.5 L179.8,322.3 L181.9,329.3 L184.0,336.3 L186.0,343.2 L188.1,350.1
 L190.1,357.0 L192.2,363.9 L194.3,370.8 L196.3,377.7 L198.4,384.6 L200.5,391.2 L202.5,397.9 L204.6,404.8
 L206.7,411.5 L208.7,418.2 L210.8,424.8 L212.9,431.3 L214.9,437.8 L217.0,444.2 L219.0,450.6 L221.1,456.8
 L223.2,462.9 L225.2,468.9 L227.3,474.9 L229.4,480.6 L231.4,486.2 L233.5,491.7 L235.6,497.1 L237.6,502.3
 L239.7,507.3 L241.7,512.2 L243.8,516.9 L245.9,521.4 L247.9,525.7 L250.0,529.7 L252.1,533.6 L254.1,537.2
 L256.2,540.6 L258.3,543.7 L260.3,546.7 L262.4,549.3 L264.4,551.7 L266.5,553.8 L268.6,555.7 L270.6,557.3
 L272.7,558.7 L274.8,559.4 L276.8,560.3 L278.9,560.9 L281.0,561.3 L283.0,561.4 L285.1,561.1 L287.1,560.6
 L289.2,560.0 L291.3,559.0 L293.3,557.8 L295.4,556.4 L297.5,554.7 L299.5,552.9 L301.6,550.8 L303.7,548.5
 L305.7,546.0 L307.8,543.3 L309.8,540.5 L311.9,537.5 L314.0,534.4 L316.0,531.1 L318.1,527.5 L320.2,523.9
 L322.2,519.9 L324.3,516.1 L326.4,512.2 L328.4,508.2 L330.5,504.1 L332.5,499.9 L334.6,495.7 L336.7,491.5
 L338.7,487.2 L340.8,482.9 L342.9,478.6 L344.9,474.3 L347.0,470.0 L349.1,465.3 L351.1,461.1 L353.2,457.0
 L355.2,452.9 L357.3,448.8 L359.4,444.8 L361.4,440.9 L363.5,437.0 L365.6,433.3 L367.6,429.6 L369.7,426.1
 L371.8,422.7 L373.8,419.4 L375.9,416.2 L377.9,413.2 L380.0,410.3 L382.1,407.6 L384.1,405.1 L386.2,402.7
 L388.3,400.5 L390.3,398.4 L392.4,396.5 L394.5,394.8 L396.5,393.3 L398.6,392.0 L400.6,390.9 L402.7,389.9
 L404.8,389.2 L406.8,388.6 L408.9,388.2 L411.0,388.0 L413.0,388.1 L415.1,388.3 L417.2,388.7 L419.2,389.2
 L421.3,390.0 L423.3,390.9 L425.4,392.1 L427.5,393.3 L429.5,394.8 L431.6,396.4 L433.7,398.2 L435.7,400.2
 L437.8,402.3 L439.9,404.5 L441.9,406.9 L444.0,409.4 L446.0,412.0 L448.1,414.7 L450.2,417.6 L452.2,420.5
 L454.3,423.6 L456.4,426.7 L458.4,430.0 L460.5,433.2 L462.6,436.6 L464.6,440.0 L466.7,443.5 L468.7,447.0
 L470.8,450.6 L472.9,454.2 L474.9,457.8 L477.0,461.4 L479.1,465.0 L481.1,468.5 L483.2,472.1 L485.3,475.8
 L487.3,479.4 L489.4,483.0 L491.4,486.5 L493.5,490.0 L495.6,493.5 L497.6,496.9 L499.7,500.2 L501.8,503.5
 L503.8,506.7 L505.9,509.9 L508.0,513.0 L510.0,516.0 L512.1,518.9 L514.2,521.8 L516.2,524.7 L518.3,527.3
 L520.3,529.9 L522.4,532.4 L524.5,534.8 L526.5,537.1 L528.6,539.3 L530.7,541.4 L532.7,543.5 L534.8,545.4
 L536.9,547.2 L538.9,548.8 L541.0,550.3 L543.0,551.8 L545.1,553.1 L547.2,554.4 L549.2,555.6 L551.3,556.6
 L553.4,557.6 L555.4,558.4 L557.5,559.1 L559.6,559.8 L561.6,560.3 L563.7,560.7 L56

In [ ]:
pltTabE1[1].XrangeMin = 0;
pltTabE1[1].XrangeMax = 2.5;
var gp = pltTabE1[1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.02 
 
 
 
 
 0.04 
 
 
 
 
 0.06 
 
 
 
 
 0.08 
 
 
 
 
 0.1 
 
 
 
 
 0.12 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta04 kineticEnergy J250 
 
 
 
 
 m2 O h01 e ta04 kineticEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,564.0 L64.3,563.7 L66.3,562.8 L68.4,561.4 L70.5,559.5 L72.5,557.2
 L74.6,554.5 L76.6,551.5 L78.7,548.2 L80.8,544.5 L82.8,540.7 L84.9,536.6 L87.0,532.2 L89.0,527.7
 L91.1,523.0 L93.2,518.1 L95.2,513.0 L97.3,507.8 L99.3,502.5 L101.4,497.0 L103.5,491.4 L105.5,485.6
 L107.6,479.7 L109.7,473.8 L111.7,467.7 L113.8,461.5 L115.9,455.2 L117.9,448.8 L120.0,442.4 L122.0,435.8
 L124.1,429.2 L126.2,422.5 L128.2,415.7 L130.3,408.9 L132.4,402.0 L134.4,395.0 L136.5,388.0 L138.6,381.0
 L140.6,373.9 L142.7,366.8 L144.7,359.6 L146.8,352.4 L148.9,345.2 L150.9,338.0 L153.0,330.8 L155.1,323.5
 L157.1,316.3 L159.2,309.0 L161.3,301.8 L163.3,294.6 L165.4,287.4 L167.4,280.2 L169.5,273.1 L171.6,266.0
 L173.6,258.9 L175.7,251.9 L177.8,244.9 L179.8,237.9 L181.9,231.1 L184.0,224.2 L186.0,217.5 L188.1,210.9
 L190.1,204.3 L192.2,197.9 L194.3,191.5 L196.3,185.3 L198.4,179.2 L200.5,173.2 L202.5,167.4 L204.6,161.8
 L206.7,156.3 L208.7,150.9 L210.8,145.8 L212.9,140.9 L214.9,136.1 L217.0,131.6 L219.0,127.4 L221.1,123.3
 L223.2,119.6 L225.2,116.1 L227.3,112.8 L229.4,109.9 L231.4,107.2 L233.5,104.9 L235.6,102.9 L237.6,101.2
 L239.7,99.9 L241.7,98.9 L243.8,98.2 L245.9,97.9 L247.9,98.0 L250.0,98.4 L252.1,99.2 L254.1,100.4
 L256.2,102.0 L258.3,103.9 L260.3,106.2 L262.4,108.9 L264.4,111.9 L266.5,115.3 L268.6,119.0 L270.6,123.1
 L272.7,127.6 L274.8,132.4 L276.8,137.5 L278.9,142.9 L281.0,148.7 L283.0,154.7 L285.1,161.1 L287.1,167.7
 L289.2,174.6 L291.3,181.7 L293.3,189.1 L295.4,196.7 L297.5,204.5 L299.5,212.5 L301.6,220.6 L303.7,229.0
 L305.7,237.5 L307.8,246.1 L309.8,254.9 L311.9,263.7 L314.0,272.6 L316.0,281.7 L318.1,290.7 L320.2,299.9
 L322.2,309.0 L324.3,318.2 L326.4,327.3 L328.4,336.5 L330.5,345.6 L332.5,354.6 L334.6,363.6 L336.7,372.5
 L338.7,381.4 L340.8,390.1 L342.9,398.8 L344.9,407.3 L347.0,415.6 L349.1,423.9 L351.1,432.0 L353.2,439.8
 L355.2,447.6 L357.3,455.1 L359.4,462.4 L361.4,469.5 L363.5,476.4 L365.6,483.1 L367.6,489.5 L369.7,495.7
 L371.8,501.6 L373.8,507.3 L375.9,512.7 L377.9,517.8 L380.0,522.7 L382.1,527.2 L384.1,531.5 L386.2,535.4
 L388.3,539.1 L390.3,542.5 L392.4,545.5 L394.5,548.3 L396.5,550.7 L398.6,552.9 L400.6,554.7 L402.7,556.3
 L404.8,557.5 L406.8,558.5 L408.9,559.2 L411.0,559.5 L413.0,559.6 L415.1,559.5 L417.2,559.0 L419.2,558.3
 L421.3,557.3 L423.3,556.1 L425.4,554.7 L427.5,553.0 L429.5,551.1 L431.6,549.0 L433.7,546.7 L435.7,544.2
 L437.8,541.5 L439.9,538.6 L441.9,535.6 L444.0,532.5 L446.0,529.2 L448.1,525.8 L450.2,522.2 L452.2,518.6
 L454.3,514.9 L456.4,511.1 L458.4,507.3 L460.5,503.4 L462.6,499.4 L464.6,495.4 L466.7,491.4 L468.7,487.4
 L470.8,483.4 L472.9,479.5 L474.9,475.5 L477.0,471.6 L479.1,467.7 L481.1,463.8 L483.2,460.1 L485.3,456.4
 L487.3,452.7 L489.4,449.2 L491.4,445.8 L493.5,442.4 L495.6,439.2 L497.6,436.1 L499.7,433.1 L501.8,430.2
 L503.8,427.4 L505.9,424.8 L508.0,422.3 L510.0,420.0 L512.1,417.8 L514.2,415.8 L516.2,413.8 L518.3,412.1
 L520.3,410.5 L522.4,409.1 L524.5,407.8 L526.5,406.6 L528.6,405.7 L530.7,404.9 L532.7,404.2 L534.8,403.7
 L536.9,403.3 L538.9,403.1 L541.0,403.1 L543.0,403.1 L545.1,403.4 L547.2,403.8 L549.2,404.3 L551.3,404.9
 L553.4,405.7 L555.4,406.6 L557.5,407.7 L559.6,408.8 L561.6,410.1 L563.7,411.5 L565.7,413.1 L567.8,414.7
 L569.9,416.4 L571.9,418.2 L574.0,420

In [ ]:
pltTabE1[2].XrangeMin = 0;
pltTabE1[2].XrangeMax = 2.5;
var gp = pltTabE1[2].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.16 
 
 
 
 
 3.18 
 
 
 
 
 3.2 
 
 
 
 
 3.22 
 
 
 
 
 3.24 
 
 
 
 
 3.26 
 
 
 
 
 3.28 
 
 
 
 
 3.3 
 
 
 
 
 3.32 
 
 
 
 
 3.34 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta04 totalEnergy J250 
 
 
 
 
 m2 O h01 e ta04 totalEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M311.0,57.1 L364.4,57.1 M62.2,91.7 L64.3,91.5 L66.3,91.4 L68.4,91.3 L70.5,91.2 L72.5,91.3
 L74.6,91.4 L76.6,91.6 L78.7,91.8 L80.8,92.2 L82.8,92.6 L84.9,93.0 L87.0,93.5 L89.0,94.1
 L91.1,94.8 L93.2,95.4 L95.2,96.2 L97.3,97.0 L99.3,97.8 L101.4,98.7 L103.5,99.6 L105.5,100.5
 L107.6,101.5 L109.7,102.6 L111.7,103.6 L113.8,104.7 L115.9,105.9 L117.9,107.0 L120.0,108.3 L122.0,109.5
 L124.1,110.8 L126.2,112.1 L128.2,113.4 L130.3,114.8 L132.4,116.2 L134.4,117.7 L136.5,119.2 L138.6,120.7
 L140.6,122.3 L142.7,123.8 L144.7,125.4 L146.8,127.1 L148.9,128.8 L150.9,130.6 L153.0,132.3 L155.1,134.1
 L157.1,136.0 L159.2,137.9 L161.3,139.8 L163.3,141.8 L165.4,143.8 L167.4,145.8 L169.5,147.9 L171.6,150.1
 L173.6,152.2 L175.7,154.4 L177.8,156.7 L179.8,158.9 L181.9,161.3 L184.0,163.7 L186.0,166.2 L188.1,168.7
 L190.1,171.2 L192.2,173.8 L194.3,176.4 L196.3,179.2 L198.4,182.0 L200.5,184.7 L202.5,187.5 L204.6,190.6
 L206.7,193.7 L208.7,196.8 L210.8,200.0 L212.9,203.2 L214.9,206.6 L217.0,210.0 L219.0,213.5 L221.1,217.0
 L223.2,220.6 L225.2,224.2 L227.3,228.1 L229.4,231.9 L231.4,235.7 L233.5,239.7 L235.6,243.7 L237.6,247.8
 L239.7,251.9 L241.7,256.1 L243.8,260.4 L245.9,264.7 L247.9,269.0 L250.0,273.3 L252.1,277.7 L254.1,282.1
 L256.2,286.5 L258.3,291.0 L260.3,295.5 L262.4,299.9 L264.4,304.3 L266.5,308.7 L268.6,313.0 L270.6,317.4
 L272.7,321.7 L274.8,325.7 L276.8,330.0 L278.9,334.2 L281.0,338.4 L283.0,342.5 L285.1,346.5 L287.1,350.4
 L289.2,354.3 L291.3,358.2 L293.3,361.9 L295.4,365.5 L297.5,369.0 L299.5,372.5 L301.6,375.9 L303.7,379.2
 L305.7,382.3 L307.8,385.3 L309.8,388.4 L311.9,391.3 L314.0,394.1 L316.0,396.8 L318.1,399.4 L320.2,401.8
 L322.2,403.9 L324.3,406.2 L326.4,408.4 L328.4,410.5 L330.5,412.4 L332.5,414.3 L334.6,416.1 L336.7,417.8
 L338.7,419.4 L340.8,420.9 L342.9,422.4 L344.9,423.8 L347.0,425.1 L349.1,425.8 L351.1,427.1 L353.2,428.2
 L355.2,429.2 L357.3,430.2 L359.4,431.1 L361.4,431.9 L363.5,432.6 L365.6,433.3 L367.6,434.0 L369.7,434.6
 L371.8,435.1 L373.8,435.6 L375.9,436.0 L377.9,436.4 L380.0,436.8 L382.1,437.1 L384.1,437.4 L386.2,437.6
 L388.3,437.8 L390.3,438.0 L392.4,438.2 L394.5,438.3 L396.5,438.5 L398.6,438.6 L400.6,438.7 L402.7,438.8
 L404.8,438.8 L406.8,438.9 L408.9,439.0 L411.0,439.1 L413.0,439.1 L415.1,439.2 L417.2,439.3 L419.2,439.4
 L421.3,439.5 L423.3,439.7 L425.4,439.8 L427.5,440.0 L429.5,440.2 L431.6,440.4 L433.7,440.6 L435.7,440.9
 L437.8,441.2 L439.9,441.6 L441.9,441.9 L444.0,442.3 L446.0,442.7 L448.1,443.2 L450.2,443.7 L452.2,444.3
 L454.3,444.8 L456.4,445.4 L458.4,446.1 L460.5,446.8 L462.6,447.5 L464.6,448.3 L466.7,449.1 L468.7,450.0
 L470.8,450.9 L472.9,451.8 L474.9,452.7 L477.0,453.7 L479.1,454.8 L481.1,455.7 L483.2,456.8 L485.3,458.0
 L487.3,459.2 L489.4,460.5 L491.4,461.7 L493.5,463.0 L495.6,464.3 L497.6,465.6 L499.7,466.9 L501.8,468.3
 L503.8,469.7 L505.9,471.1 L508.0,472.5 L510.0,474.0 L512.1,475.5 L514.2,476.9 L516.2,478.6 L518.3,480.0
 L520.3,481.6 L522.4,483.1 L524.5,484.6 L526.5,486.2 L528.6,487.8 L530.7,489.3 L532.7,490.9 L534.8,492.5
 L536.9,494.0 L538.9,495.5 L541.0,497.0 L543.0,498.5 L545.1,500.0 L547.2,501.6 L549.2,503.1 L551.3,504.6
 L553.4,506.0 L555.4,507.5 L557.5,508.9 L559.6,510.3 L561.6,511.7 L563.7,513.1 L565

In [ ]:
// for(int i = 0; i < 4; i++) {
//     Plot2Ddata dat = new Plot2Ddata();
//     var grp = PlotTableDM[i,0].dataGroups[0];
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

In [ ]:
Plot2Ddata[] pltTabE2 = PlotTableEnrgy[1];

In [ ]:
pltTabE2[0].XrangeMin = 0;
pltTabE2[0].XrangeMax = 2.5;
var gp = pltTabE2[0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.05 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0.3 
 
 
 
 
 0.35 
 
 
 
 
 0.4 
 
 
 
 
 0.45 
 
 
 
 
 0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 surfaceEnergy J250 
 
 
 
 
 m2 O h01 e ta07 surfaceEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,71.9 L64.3,71.9 L66.3,72.6 L68.4,73.7 L70.5,75.3 L72.5,77.1
 L74.6,79.2 L76.6,81.4 L78.7,83.7 L80.8,86.1 L82.8,88.6 L84.9,91.1 L87.0,93.7 L89.0,96.3
 L91.1,99.0 L93.2,101.7 L95.2,104.5 L97.3,119.5 L99.3,116.7 L101.4,112.9 L103.5,115.8 L105.5,118.7
 L107.6,121.6 L109.7,124.5 L111.7,127.4 L113.8,130.4 L115.9,133.4 L117.9,136.4 L120.0,139.7 L122.0,142.6
 L124.1,145.5 L126.2,148.8 L128.2,151.8 L130.3,154.8 L132.4,157.9 L134.4,161.1 L136.5,164.2 L138.6,167.5
 L140.6,170.8 L142.7,174.3 L144.7,177.7 L146.8,181.1 L148.9,184.5 L150.9,187.9 L153.0,191.3 L155.1,194.8
 L157.1,198.3 L159.2,201.9 L161.3,205.5 L163.3,209.1 L165.4,212.7 L167.4,216.4 L169.5,220.2 L171.6,223.9
 L173.6,227.7 L175.7,231.4 L177.8,235.3 L179.8,239.2 L181.9,243.1 L184.0,247.0 L186.0,251.0 L188.1,254.9
 L190.1,258.9 L192.2,262.9 L194.3,266.9 L196.3,271.0 L198.4,275.0 L200.5,279.1 L202.5,283.1 L204.6,287.2
 L206.7,291.3 L208.7,295.4 L210.8,299.6 L212.9,303.6 L214.9,307.8 L217.0,311.9 L219.0,316.1 L221.1,320.3
 L223.2,324.5 L225.2,328.6 L227.3,332.8 L229.4,337.0 L231.4,341.2 L233.5,345.5 L235.6,349.7 L237.6,353.9
 L239.7,358.2 L241.7,362.5 L243.8,366.8 L245.9,370.9 L247.9,375.2 L250.0,379.6 L252.1,383.8 L254.1,388.1
 L256.2,392.4 L258.3,396.7 L260.3,401.2 L262.4,405.5 L264.4,409.8 L266.5,414.1 L268.6,418.8 L270.6,423.0
 L272.7,427.1 L274.8,431.3 L276.8,435.4 L278.9,439.2 L281.0,443.1 L283.0,447.6 L285.1,451.6 L287.1,455.5
 L289.2,459.6 L291.3,463.5 L293.3,467.2 L295.4,470.9 L297.5,474.4 L299.5,477.7 L301.6,480.9 L303.7,483.9
 L305.7,486.9 L307.8,490.3 L309.8,492.9 L311.9,495.5 L314.0,497.9 L316.0,500.1 L318.1,502.2 L320.2,504.1
 L322.2,505.9 L324.3,507.5 L326.4,508.9 L328.4,510.5 L330.5,511.5 L332.5,512.4 L334.6,513.0 L336.7,513.5
 L338.7,513.7 L340.8,513.8 L342.9,513.9 L344.9,514.4 L347.0,513.9 L349.1,513.3 L351.1,512.4 L353.2,511.4
 L355.2,510.2 L357.3,509.0 L359.4,507.5 L361.4,505.8 L363.5,504.0 L365.6,501.9 L367.6,499.7 L369.7,497.6
 L371.8,495.2 L373.8,492.7 L375.9,490.1 L377.9,487.5 L380.0,484.0 L382.1,481.2 L384.1,478.2 L386.2,475.3
 L388.3,472.3 L390.3,469.3 L392.4,466.3 L394.5,463.2 L396.5,460.2 L398.6,457.2 L400.6,454.1 L402.7,451.1
 L404.8,448.1 L406.8,445.1 L408.9,443.5 L411.0,440.5 L413.0,437.5 L415.1,434.6 L417.2,431.7 L419.2,428.9
 L421.3,426.2 L423.3,423.6 L425.4,421.1 L427.5,418.7 L429.5,416.4 L431.6,414.2 L433.7,412.1 L435.7,410.2
 L437.8,408.3 L439.9,406.6 L441.9,405.0 L444.0,403.5 L446.0,402.1 L448.1,400.9 L450.2,399.8 L452.2,398.8
 L454.3,398.0 L456.4,397.3 L458.4,396.8 L460.5,396.4 L462.6,396.1 L464.6,396.0 L466.7,396.1 L468.7,396.3
 L470.8,396.6 L472.9,397.1 L474.9,397.7 L477.0,398.4 L479.1,399.3 L481.1,400.3 L483.2,401.4 L485.3,402.6
 L487.3,403.9 L489.4,405.4 L491.4,406.9 L493.5,408.6 L495.6,410.3 L497.6,412.2 L499.7,414.1 L501.8,416.1
 L503.8,418.1 L505.9,420.3 L508.0,422.5 L510.0,424.7 L512.1,427.0 L514.2,429.4 L516.2,431.8 L518.3,434.2
 L520.3,436.8 L522.4,439.4 L524.5,441.9 L526.5,444.4 L528.6,447.0 L530.7,449.5 L532.7,452.0 L534.8,454.5
 L536.9,457.1 L538.9,459.6 L541.0,462.1 L543.0,464.6 L545.1,467.1 L547.2,469.5 L549.2,471.9 L551.3,474.3
 L553.4,476.6 L555.4,478.8 L

In [ ]:
pltTabE2[1].XrangeMin = 0;
pltTabE2[1].XrangeMax = 2.5;
var gp = pltTabE2[1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 kineticEnergy J250 
 
 
 
 
 m2 O h01 e ta07 kineticEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,564.0 L64.3,563.4 L66.3,561.8 L68.4,559.6 L70.5,556.9 L72.5,554.0
 L74.6,551.0 L76.6,547.8 L78.7,544.7 L80.8,541.5 L82.8,538.2 L84.9,534.9 L87.0,531.5 L89.0,528.1
 L91.1,524.7 L93.2,521.1 L95.2,517.6 L97.3,514.0 L99.3,510.4 L101.4,506.8 L103.5,503.0 L105.5,499.3
 L107.6,495.5 L109.7,491.7 L111.7,487.9 L113.8,484.0 L115.9,480.1 L117.9,476.1 L120.0,472.1 L122.0,468.0
 L124.1,463.8 L126.2,459.7 L128.2,455.4 L130.3,451.1 L132.4,446.7 L134.4,442.3 L136.5,437.8 L138.6,433.3
 L140.6,428.6 L142.7,424.0 L144.7,419.2 L146.8,414.4 L148.9,409.5 L150.9,404.6 L153.0,399.6 L155.1,394.6
 L157.1,389.5 L159.2,384.4 L161.3,379.2 L163.3,374.0 L165.4,368.7 L167.4,363.3 L169.5,358.0 L171.6,352.6
 L173.6,347.1 L175.7,341.6 L177.8,336.1 L179.8,330.6 L181.9,325.0 L184.0,319.4 L186.0,313.8 L188.1,308.2
 L190.1,302.6 L192.2,296.9 L194.3,291.3 L196.3,285.6 L198.4,280.0 L200.5,274.3 L202.5,268.7 L204.6,263.1
 L206.7,257.4 L208.7,251.8 L210.8,246.2 L212.9,240.6 L214.9,235.0 L217.0,229.5 L219.0,223.9 L221.1,218.4
 L223.2,212.8 L225.2,207.3 L227.3,201.8 L229.4,196.3 L231.4,190.8 L233.5,185.3 L235.6,179.8 L237.6,174.4
 L239.7,168.9 L241.7,163.5 L243.8,158.1 L245.9,152.7 L247.9,147.4 L250.0,142.2 L252.1,137.1 L254.1,132.0
 L256.2,127.1 L258.3,122.2 L260.3,117.5 L262.4,112.9 L264.4,108.4 L266.5,104.1 L268.6,99.9 L270.6,95.9
 L272.7,92.0 L274.8,88.4 L276.8,84.9 L278.9,81.8 L281.0,78.8 L283.0,76.1 L285.1,73.7 L287.1,71.6
 L289.2,69.8 L291.3,68.1 L293.3,66.8 L295.4,65.9 L297.5,65.4 L299.5,65.1 L301.6,65.3 L303.7,65.7
 L305.7,66.6 L307.8,67.9 L309.8,69.5 L311.9,71.5 L314.0,73.8 L316.0,76.6 L318.1,79.8 L320.2,83.4
 L322.2,87.3 L324.3,91.7 L326.4,96.4 L328.4,101.5 L330.5,106.9 L332.5,112.7 L334.6,118.8 L336.7,125.2
 L338.7,132.0 L340.8,139.1 L342.9,146.4 L344.9,154.1 L347.0,162.0 L349.1,170.1 L351.1,178.4 L353.2,187.0
 L355.2,195.8 L357.3,204.8 L359.4,213.9 L361.4,223.2 L363.5,232.6 L365.6,242.2 L367.6,251.8 L369.7,261.5
 L371.8,271.2 L373.8,281.0 L375.9,290.7 L377.9,300.5 L380.0,310.3 L382.1,319.9 L384.1,329.6 L386.2,339.2
 L388.3,348.8 L390.3,358.3 L392.4,367.7 L394.5,377.0 L396.5,386.2 L398.6,395.3 L400.6,404.2 L402.7,412.9
 L404.8,421.5 L406.8,429.9 L408.9,438.1 L411.0,446.2 L413.0,453.9 L415.1,461.5 L417.2,468.9 L419.2,476.0
 L421.3,482.9 L423.3,489.5 L425.4,495.9 L427.5,501.9 L429.5,507.7 L431.6,513.3 L433.7,518.5 L435.7,523.4
 L437.8,528.0 L439.9,532.3 L441.9,536.3 L444.0,540.0 L446.0,543.4 L448.1,546.4 L450.2,549.1 L452.2,551.5
 L454.3,553.6 L456.4,555.3 L458.4,556.7 L460.5,557.9 L462.6,558.7 L464.6,559.2 L466.7,559.4 L468.7,559.3
 L470.8,558.9 L472.9,558.3 L474.9,557.3 L477.0,556.2 L479.1,554.7 L481.1,553.0 L483.2,551.1 L485.3,549.0
 L487.3,546.6 L489.4,544.0 L491.4,541.3 L493.5,538.4 L495.6,535.3 L497.6,532.1 L499.7,528.7 L501.8,525.2
 L503.8,521.6 L505.9,517.9 L508.0,514.1 L510.0,510.2 L512.1,506.3 L514.2,502.3 L516.2,498.3 L518.3,494.3
 L520.3,490.3 L522.4,486.2 L524.5,482.2 L526.5,478.2 L528.6,474.2 L530.7,470.3 L532.7,466.4 L534.8,462.6
 L536.9,458.9 L538.9,455.2 L541.0,451.7 L543.0,448.2 L545.1,444.9 L547.2,441.7 L549.2,438.5 L551.3,435.6
 L553.4,432.7 L555.4,430.0 L557.5,427.4 L559.6,425.0 L561.6,422.7 L563.7,420.6 L565.7,418.6 L567.8,416.8
 L569.9,415.1 L571.9,413.5 L574.0,412.1 L576.1,411.0 L578.1,410.0 L580.2,409.2 L582.3

In [ ]:
pltTabE2[2].XrangeMin = 0;
pltTabE2[2].XrangeMax = 2.5;
var gp = pltTabE2[2].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.15 
 
 
 
 
 3.2 
 
 
 
 
 3.25 
 
 
 
 
 3.3 
 
 
 
 
 3.35 
 
 
 
 
 3.4 
 
 
 
 
 3.45 
 
 
 
 
 3.5 
 
 
 
 
 3.55 
 
 
 
 
 3.6 
 
 
 
 
 3.65 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 totalEnergy J250 
 
 
 
 
 m2 O h01 e ta07 totalEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M311.0,57.1 L364.4,57.1 M62.2,84.4 L64.3,84.1 L66.3,84.0 L68.4,84.1 L70.5,84.5 L72.5,85.1
 L74.6,85.8 L76.6,86.7 L78.7,87.6 L80.8,88.7 L82.8,89.8 L84.9,90.9 L87.0,92.1 L89.0,93.3
 L91.1,94.5 L93.2,95.7 L95.2,97.0 L97.3,111.7 L99.3,106.9 L101.4,100.8 L103.5,102.1 L105.5,103.4
 L107.6,104.8 L109.7,106.1 L111.7,107.4 L113.8,108.7 L115.9,110.0 L117.9,111.3 L120.0,113.0 L122.0,114.1
 L124.1,115.3 L126.2,116.8 L128.2,117.9 L130.3,119.1 L132.4,120.3 L134.4,121.6 L136.5,122.8 L138.6,124.1
 L140.6,125.4 L142.7,127.0 L144.7,128.4 L146.8,129.7 L148.9,131.0 L150.9,132.3 L153.0,133.6 L155.1,134.9
 L157.1,136.2 L159.2,137.5 L161.3,138.9 L163.3,140.2 L165.4,141.6 L167.4,143.0 L169.5,144.4 L171.6,145.8
 L173.6,147.3 L175.7,148.7 L177.8,150.2 L179.8,151.7 L181.9,153.2 L184.0,154.7 L186.0,156.3 L188.1,157.8
 L190.1,159.4 L192.2,160.9 L194.3,162.5 L196.3,164.1 L198.4,165.8 L200.5,167.4 L202.5,169.1 L204.6,170.7
 L206.7,172.4 L208.7,174.1 L210.8,175.9 L212.9,177.6 L214.9,179.3 L217.0,181.1 L219.0,183.0 L221.1,184.8
 L223.2,186.6 L225.2,188.4 L227.3,190.3 L229.4,192.1 L231.4,194.0 L233.5,195.9 L235.6,197.8 L237.6,199.8
 L239.7,201.7 L241.7,203.7 L243.8,205.7 L245.9,207.6 L247.9,209.7 L250.0,211.9 L252.1,214.0 L254.1,216.2
 L256.2,218.5 L258.3,220.8 L260.3,223.3 L262.4,225.7 L264.4,228.3 L266.5,230.8 L268.6,233.9 L270.6,236.5
 L272.7,239.1 L274.8,241.9 L276.8,244.7 L278.9,247.3 L281.0,250.1 L283.0,253.7 L285.1,256.9 L287.1,260.1
 L289.2,263.8 L291.3,267.1 L293.3,270.6 L295.4,274.3 L297.5,277.7 L299.5,281.3 L301.6,284.8 L303.7,288.5
 L305.7,292.2 L307.8,296.5 L309.8,300.2 L311.9,304.0 L314.0,307.9 L316.0,311.7 L318.1,315.6 L320.2,319.5
 L322.2,323.4 L324.3,327.3 L326.4,331.2 L328.4,335.5 L330.5,339.4 L332.5,343.2 L334.6,347.0 L336.7,350.7
 L338.7,354.4 L340.8,358.0 L342.9,361.7 L344.9,366.1 L347.0,369.5 L349.1,372.9 L351.1,376.2 L353.2,379.3
 L355.2,382.4 L357.3,385.5 L359.4,388.5 L361.4,391.3 L363.5,394.0 L365.6,396.5 L367.6,398.9 L369.7,401.3
 L371.8,403.6 L373.8,405.7 L375.9,407.8 L377.9,409.8 L380.0,410.8 L382.1,412.5 L384.1,414.1 L386.2,415.7
 L388.3,417.2 L390.3,418.6 L392.4,420.0 L394.5,421.3 L396.5,422.6 L398.6,423.8 L400.6,424.9 L402.7,425.9
 L404.8,426.9 L406.8,427.8 L408.9,430.2 L411.0,430.8 L413.0,431.5 L415.1,432.0 L417.2,432.6 L419.2,433.0
 L421.3,433.5 L423.3,434.0 L425.4,434.4 L427.5,434.8 L429.5,435.2 L431.6,435.5 L433.7,435.9 L435.7,436.2
 L437.8,436.4 L439.9,436.7 L441.9,436.9 L444.0,437.1 L446.0,437.3 L448.1,437.4 L450.2,437.6 L452.2,437.7
 L454.3,437.8 L456.4,438.0 L458.4,438.1 L460.5,438.2 L462.6,438.4 L464.6,438.5 L466.7,438.7 L468.7,438.8
 L470.8,439.0 L472.9,439.2 L474.9,439.4 L477.0,439.6 L479.1,439.8 L481.1,440.1 L483.2,440.3 L485.3,440.6
 L487.3,440.9 L489.4,441.2 L491.4,441.5 L493.5,441.9 L495.6,442.3 L497.6,442.7 L499.7,443.1 L501.8,443.5
 L503.8,444.0 L505.9,444.5 L508.0,445.0 L510.0,445.6 L512.1,446.1 L514.2,446.7 L516.2,447.4 L518.3,448.0
 L520.3,448.9 L522.4,449.7 L524.5,450.5 L526.5,451.2 L528.6,452.0 L530.7,452.8 L532.7,453.7 L534.8,454.6
 L536.9,455.5 L538.9,456.4 L541.0,457.4 L543.0,458.4 L545.1,459.5 L547.2,460.5 L549.2,461.6 L551.3,462.7
 L553.4,463.9 L555.4,465.0 L557.5,466.1 L559.6,467.3 L561

## m=3 cases (eta = {0.4})

In [ ]:
Plot2Ddata[] pltTabE3 = PlotTableEnrgy[2];

In [ ]:
pltTabE3[0].XrangeMin = 0;
pltTabE3[0].XrangeMax = 1.0;
var gp = pltTabE3[0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0.3 
 
 
 
 
 0.35 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m3Oh01eta04 surfaceEnergy J250 
 
 
 
 
 m3 O h01 e ta04 surfaceEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,81.9 L65.6,81.9 L69.1,82.7 L72.5,84.2 L76.0,86.5 L79.4,89.4
 L82.8,93.0 L86.3,97.1 L89.7,101.9 L93.2,107.0 L96.6,112.6 L100.0,118.6 L103.5,124.9 L106.9,131.6
 L110.4,138.6 L113.8,146.0 L117.2,153.6 L120.7,161.4 L124.1,169.5 L127.6,177.8 L131.0,186.3 L134.4,195.0
 L137.9,203.8 L141.3,212.8 L144.7,221.9 L148.2,231.2 L151.6,240.5 L155.1,249.9 L158.5,259.3 L161.9,268.8
 L165.4,278.3 L168.8,287.7 L172.3,297.2 L175.7,306.6 L179.1,316.0 L182.6,325.3 L186.0,334.6 L189.5,343.8
 L192.9,353.0 L196.3,361.8 L199.8,370.6 L203.2,379.2 L206.7,388.0 L210.1,396.2 L213.5,404.4 L217.0,412.6
 L220.4,420.7 L223.9,428.9 L227.3,435.8 L230.7,443.0 L234.2,450.1 L237.6,457.0 L241.1,463.5 L244.5,469.8
 L247.9,475.9 L251.4,481.7 L254.8,487.3 L258.3,492.7 L261.7,497.9 L265.1,502.6 L268.6,507.3 L272.0,511.5
 L275.4,515.7 L278.9,519.6 L282.3,523.2 L285.8,526.5 L289.2,529.6 L292.6,532.5 L296.1,535.2 L299.5,537.6
 L303.0,539.4 L306.4,541.2 L309.8,543.2 L313.3,544.7 L316.7,545.9 L320.2,546.9 L323.6,547.7 L327.0,548.2
 L330.5,548.5 L333.9,548.6 L337.4,548.0 L340.8,547.9 L344.2,547.4 L347.7,545.8 L351.1,545.2 L354.6,544.2
 L358.0,543.0 L361.4,541.6 L364.9,540.1 L368.3,538.4 L371.8,536.5 L375.2,534.5 L378.6,532.4 L382.1,530.1
 L385.5,527.7 L389.0,525.1 L392.4,522.5 L395.8,519.9 L399.3,517.1 L402.7,514.3 L406.2,511.5 L409.6,508.6
 L413.0,505.6 L416.5,502.1 L419.9,499.3 L423.3,496.4 L426.8,493.5 L430.2,490.6 L433.7,487.7 L437.1,484.8
 L440.5,482.0 L444.0,478.9 L447.4,476.3 L450.9,473.7 L454.3,471.0 L457.7,468.6 L461.2,466.3 L464.6,464.2
 L468.1,462.2 L471.5,460.1 L474.9,458.1 L478.4,456.4 L481.8,454.8 L485.3,453.3 L488.7,451.9 L492.1,450.8
 L495.6,449.7 L499.0,448.9 L502.5,448.2 L505.9,447.6 L509.3,447.2 L512.8,446.9 L516.2,446.8 L519.7,446.8
 L523.1,447.0 L526.5,447.4 L530.0,447.9 L533.4,448.5 L536.9,449.2 L540.3,450.1 L543.7,451.2 L547.2,452.3
 L550.6,453.6 L554.0,454.9 L557.5,456.4 L560.9,458.0 L564.4,459.7 L567.8,461.5 L571.2,463.3 L574.7,465.2
 L578.1,467.2 L581.6,469.3 L585.0,470.8 L588.4,472.9 L591.9,475.6 L595.3,477.9 L598.8,480.2 L602.2,481.8
 L605.6,484.8 L609.1,482.7 L612.5,484.8 L616.0,491.7 L619.4,494.1 L622.8,496.5 L626.3,497.7 L629.7,499.9
 L633.2,503.1 L636.6,505.6 L640.0,507.7 L643.5,509.2 L646.9,511.6 L650.4,515.9 L653.8,516.1 L657.2,520.4
 L660.7,522.8 L664.1,524.9 L667.6,527.0 L671.0,529.0 L674.4,531.0 L677.9,531.3 L681.3,533.4 L684.7,535.4
 L688.2,538.6 L691.6,540.6 L695.1,542.3 L698.5,544.0 L701.9,545.5 L705.4,547.0 L708.8,548.4 L712.3,549.8
 L715.7,551.1 L719.1,552.3 L722.6,553.4 L726.0,554.4 L729.5,555.3 L732.9,556.2 L736.3,557.0 L739.8,557.8
 L743.2,558.4 L746.7,559.0 L750.1,559.5 '/> 
 
 m3Oh01eta04 surfaceEnergy J250-dtmin 
 
 
 m3 O h01 e ta04 surfaceEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M288.8,81.1 L342.2,81.1 M62.2,81.9 L63.2,81.9 L64.3,81.9 L65.3,82.0 L66.3,82.2 L67.4,82.4
 L68.4,82.8 L69.4,83.2 L70.5,83.7 L71.5,84.2 L72.5,84.9 L73.6,85.6 L74.6,86.3 L75.6,87.2
 L76.6,88.0 L77.7,89.0 L78.7,90.0 L79.7,91.0 L80.8,92.1 L81.8,93.3 L82.8,94.5 L83.9,95.9
 L84.9,97.2 L85.9,98.5 L87.0,99.9 L88.0,101.3 L89.0,102.7 L90.1,104.3 L91.1,105.8 L92.1,107.4
 L93.2,109.0 L94.2,110.7 L95.2,112.4 L96.3,114.1 L97.3,115.9 L98.3,117.7 

In [ ]:
pltTabE3[1].XrangeMin = 0;
pltTabE3[1].XrangeMax = 1.0;
var gp = pltTabE3[1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.02 
 
 
 
 
 0.04 
 
 
 
 
 0.06 
 
 
 
 
 0.08 
 
 
 
 
 0.1 
 
 
 
 
 0.12 
 
 
 
 
 0.14 
 
 
 
 
 0.16 
 
 
 
 
 0.18 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m3Oh01eta04 kineticEnergy J250 
 
 
 
 
 m3 O h01 e ta04 kineticEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,564.0 L65.6,563.3 L69.1,561.1 L72.5,557.7 L76.0,553.2 L79.4,547.8
 L82.8,541.5 L86.3,534.5 L89.7,526.8 L93.2,518.4 L96.6,509.6 L100.0,500.2 L103.5,490.5 L106.9,480.3
 L110.4,469.8 L113.8,459.0 L117.2,447.9 L120.7,436.6 L124.1,425.1 L127.6,413.3 L131.0,401.5 L134.4,389.5
 L137.9,377.5 L141.3,365.4 L144.7,353.3 L148.2,341.2 L151.6,329.1 L155.1,317.2 L158.5,305.4 L161.9,293.7
 L165.4,282.2 L168.8,270.9 L172.3,259.8 L175.7,249.0 L179.1,238.4 L182.6,228.2 L186.0,218.2 L189.5,208.6
 L192.9,199.2 L196.3,190.3 L199.8,181.7 L203.2,173.6 L206.7,165.8 L210.1,158.4 L213.5,151.5 L217.0,145.0
 L220.4,138.9 L223.9,133.2 L227.3,128.0 L230.7,123.2 L234.2,119.0 L237.6,115.2 L241.1,111.9 L244.5,109.0
 L247.9,106.7 L251.4,104.8 L254.8,103.4 L258.3,102.5 L261.7,102.0 L265.1,102.1 L268.6,102.5 L272.0,103.5
 L275.4,104.9 L278.9,106.8 L282.3,109.0 L285.8,111.8 L289.2,114.9 L292.6,118.5 L296.1,122.4 L299.5,126.8
 L303.0,131.6 L306.4,136.8 L309.8,142.2 L313.3,148.0 L316.7,154.2 L320.2,160.6 L323.6,167.4 L327.0,174.6
 L330.5,182.0 L333.9,189.6 L337.4,197.6 L340.8,205.8 L344.2,214.2 L347.7,222.9 L351.1,231.7 L354.6,240.7
 L358.0,249.8 L361.4,259.1 L364.9,268.6 L368.3,278.1 L371.8,287.7 L375.2,297.4 L378.6,307.2 L382.1,317.0
 L385.5,326.8 L389.0,336.6 L392.4,346.3 L395.8,356.0 L399.3,365.7 L402.7,375.3 L406.2,384.7 L409.6,394.0
 L413.0,403.2 L416.5,412.2 L419.9,421.0 L423.3,429.6 L426.8,438.0 L430.2,446.2 L433.7,454.1 L437.1,461.8
 L440.5,469.2 L444.0,476.3 L447.4,483.2 L450.9,489.8 L454.3,496.0 L457.7,502.0 L461.2,507.6 L464.6,512.9
 L468.1,517.9 L471.5,522.5 L474.9,526.7 L478.4,530.7 L481.8,534.3 L485.3,537.7 L488.7,540.6 L492.1,543.3
 L495.6,545.6 L499.0,547.7 L502.5,549.4 L505.9,550.8 L509.3,551.9 L512.8,552.7 L516.2,553.2 L519.7,553.4
 L523.1,553.4 L526.5,553.1 L530.0,552.6 L533.4,551.9 L536.9,550.9 L540.3,549.8 L543.7,548.4 L547.2,546.8
 L550.6,545.1 L554.0,543.2 L557.5,541.2 L560.9,539.0 L564.4,536.7 L567.8,534.3 L571.2,531.8 L574.7,529.2
 L578.1,526.5 L581.6,523.7 L585.0,521.0 L588.4,518.1 L591.9,515.3 L595.3,512.3 L598.8,509.4 L602.2,506.6
 L605.6,503.7 L609.1,500.8 L612.5,497.8 L616.0,495.0 L619.4,492.2 L622.8,489.3 L626.3,486.6 L629.7,483.9
 L633.2,481.2 L636.6,478.6 L640.0,476.0 L643.5,473.6 L646.9,471.2 L650.4,468.9 L653.8,466.7 L657.2,464.6
 L660.7,462.6 L664.1,460.7 L667.6,458.8 L671.0,457.2 L674.4,455.6 L677.9,454.2 L681.3,452.9 L684.7,451.7
 L688.2,450.6 L691.6,449.6 L695.1,448.6 L698.5,447.9 L701.9,447.3 L705.4,446.8 L708.8,446.4 L712.3,446.1
 L715.7,446.0 L719.1,446.0 L722.6,446.2 L726.0,446.4 L729.5,446.8 L732.9,447.4 L736.3,448.0 L739.8,448.7
 L743.2,449.6 L746.7,450.5 L750.1,451.6 '/> 
 
 m3Oh01eta04 kineticEnergy J250-dtmin 
 
 
 m3 O h01 e ta04 kineticEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M288.8,81.1 L342.2,81.1 M62.2,564.0 L63.2,563.9 L64.3,563.7 L65.3,563.4 L66.3,562.9 L67.4,562.3
 L68.4,561.6 L69.4,560.7 L70.5,559.8 L71.5,558.8 L72.5,557.6 L73.6,556.4 L74.6,555.1 L75.6,553.6
 L76.6,552.1 L77.7,550.6 L78.7,548.9 L79.7,547.2 L80.8,545.3 L81.8,543.5 L82.8,541.5 L83.9,539.5
 L84.9,537.4 L85.9,535.2 L87.0,533.0 L88.0,530.8 L89.0,528.4 L90.1,526.1 L91.1,523.6 

In [ ]:
pltTabE3[2].XrangeMin = 0;
pltTabE3[2].XrangeMax = 1.0;
var gp = pltTabE3[2].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.15 
 
 
 
 
 3.2 
 
 
 
 
 3.25 
 
 
 
 
 3.3 
 
 
 
 
 3.35 
 
 
 
 
 3.4 
 
 
 
 
 3.45 
 
 
 
 
 3.5 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m3Oh01eta04 totalEnergy J250 
 
 
 
 
 m3 O h01 e ta04 totalEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M311.0,57.1 L364.4,57.1 M62.2,94.6 L65.6,94.2 L69.1,93.9 L72.5,93.6 L76.0,93.6 L79.4,93.7
 L82.8,94.1 L86.3,94.6 L89.7,95.4 L93.2,96.3 L96.6,97.3 L100.0,98.4 L103.5,99.8 L106.9,101.3
 L110.4,102.9 L113.8,104.7 L117.2,106.5 L120.7,108.6 L124.1,110.7 L127.6,113.0 L131.0,115.4 L134.4,117.9
 L137.9,120.6 L141.3,123.3 L144.7,126.2 L148.2,129.3 L151.6,132.4 L155.1,135.6 L158.5,139.0 L161.9,142.5
 L165.4,146.0 L168.8,149.7 L172.3,153.4 L175.7,157.3 L179.1,161.2 L182.6,165.2 L186.0,169.5 L189.5,173.6
 L192.9,178.0 L196.3,182.3 L199.8,186.7 L203.2,191.1 L206.7,195.8 L210.1,200.3 L213.5,204.9 L217.0,209.8
 L220.4,214.7 L223.9,220.0 L227.3,224.2 L230.7,229.0 L234.2,234.0 L237.6,238.8 L241.1,243.7 L244.5,248.5
 L247.9,253.4 L251.4,258.3 L254.8,263.1 L258.3,268.1 L261.7,273.0 L265.1,277.7 L268.6,282.7 L272.0,287.4
 L275.4,292.3 L278.9,297.1 L282.3,301.9 L285.8,306.6 L289.2,311.4 L292.6,316.1 L296.1,320.8 L299.5,325.4
 L303.0,329.7 L306.4,334.2 L309.8,339.0 L313.3,343.5 L316.7,347.8 L320.2,352.2 L323.6,356.4 L327.0,360.6
 L330.5,364.7 L333.9,368.7 L337.4,372.3 L340.8,376.3 L344.2,380.2 L347.7,383.1 L351.1,387.0 L354.6,390.6
 L358.0,394.1 L361.4,397.5 L364.9,400.8 L368.3,404.0 L371.8,407.1 L375.2,410.1 L378.6,413.0 L382.1,415.7
 L385.5,418.4 L389.0,420.8 L392.4,423.2 L395.8,425.7 L399.3,427.8 L402.7,429.9 L406.2,431.9 L409.6,433.8
 L413.0,435.6 L416.5,436.7 L419.9,438.5 L423.3,440.0 L426.8,441.4 L430.2,442.7 L433.7,443.9 L437.1,444.9
 L440.5,445.9 L444.0,446.5 L447.4,447.4 L450.9,448.2 L454.3,448.8 L457.7,449.4 L461.2,449.9 L464.6,450.6
 L468.1,451.1 L471.5,451.4 L474.9,451.6 L478.4,451.9 L481.8,452.2 L485.3,452.4 L488.7,452.6 L492.1,452.8
 L495.6,453.0 L499.0,453.2 L502.5,453.3 L505.9,453.5 L509.3,453.6 L512.8,453.8 L516.2,453.9 L519.7,454.1
 L523.1,454.3 L526.5,454.5 L530.0,454.7 L533.4,454.9 L536.9,455.2 L540.3,455.5 L543.7,455.8 L547.2,456.2
 L550.6,456.5 L554.0,456.9 L557.5,457.4 L560.9,457.8 L564.4,458.3 L567.8,458.9 L571.2,459.4 L574.7,460.0
 L578.1,460.6 L581.6,461.3 L585.0,461.3 L588.4,462.0 L591.9,463.2 L595.3,464.0 L598.8,464.9 L602.2,464.9
 L605.6,466.5 L609.1,462.9 L612.5,463.4 L616.0,468.9 L619.4,469.9 L622.8,470.8 L626.3,470.6 L629.7,471.4
 L633.2,473.2 L636.6,474.3 L640.0,475.2 L643.5,475.4 L646.9,476.6 L650.4,479.7 L653.8,478.7 L657.2,481.9
 L660.7,483.3 L664.1,484.5 L667.6,485.6 L671.0,486.7 L674.4,487.9 L677.9,487.5 L681.3,489.0 L684.7,490.3
 L688.2,492.9 L691.6,494.4 L695.1,495.7 L698.5,496.9 L701.9,498.2 L705.4,499.4 L708.8,500.6 L712.3,501.9
 L715.7,503.1 L719.1,504.3 L722.6,505.5 L726.0,506.6 L729.5,507.8 L732.9,508.9 L736.3,510.0 L739.8,511.2
 L743.2,512.3 L746.7,513.3 L750.1,514.4 '/> 
 
 m3Oh01eta04 totalEnergy J250-dtmin 
 
 
 m3 O h01 e ta04 totalEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M311.0,81.1 L364.4,81.1 M62.2,94.6 L63.2,94.5 L64.3,94.5 L65.3,94.4 L66.3,94.3 L67.4,94.2
 L68.4,94.2 L69.4,94.2 L70.5,94.2 L71.5,94.2 L72.5,94.3 L73.6,94.3 L74.6,94.4 L75.6,94.5
 L76.6,94.6 L77.7,94.7 L78.7,94.9 L79.7,95.0 L80.8,95.2 L81.8,95.4 L82.8,95.6 L83.9,95.9
 L84.9,96.2 L85.9,96.4 L87.0,96.6 L88.0,96.9 L89.0,97.1 L90.1,97.4 L91.1,97.7 L92.1,98.0
 L93.2,98.3 L94.2,98.7 L95.2,99.0 L96.3,99.4 L97.3,99.8 L98.3,100.2 L99.3,100.6 L100.4,10

## m=4 cases (eta = {0.4, 0.7})

In [ ]:
Plot2Ddata[] pltTabE4 = PlotTableEnrgy[3];

In [ ]:
pltTabE4[0].XrangeMin = 0;
pltTabE4[0].XrangeMax = 1.0;
var gp = pltTabE4[0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0.3 
 
 
 
 
 0.35 
 
 
 
 
 0.4 
 
 
 
 
 0.45 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta04 surfaceEnergy J250 
 
 
 
 
 m4 O h01 e ta04 surfaceEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,49.2 L65.6,49.1 L69.1,50.5 L72.5,52.9 L76.0,56.4 L79.4,60.9
 L82.8,66.3 L86.3,72.6 L89.7,79.6 L93.2,87.3 L96.6,95.8 L100.0,104.9 L103.5,114.7 L106.9,124.9
 L110.4,135.6 L113.8,146.8 L117.2,158.3 L120.7,170.3 L124.1,182.6 L127.6,195.2 L131.0,208.0 L134.4,221.0
 L137.9,234.2 L141.3,247.4 L144.7,260.7 L148.2,274.1 L151.6,287.4 L155.1,300.6 L158.5,313.8 L161.9,328.0
 L165.4,341.1 L168.8,353.3 L172.3,366.6 L175.7,378.9 L179.1,390.9 L182.6,402.5 L186.0,413.7 L189.5,424.4
 L192.9,434.7 L196.3,444.6 L199.8,454.6 L203.2,463.4 L206.7,471.6 L210.1,479.6 L213.5,486.6 L217.0,493.2
 L220.4,499.3 L223.9,505.0 L227.3,510.4 L230.7,515.5 L234.2,520.1 L237.6,524.3 L241.1,529.7 L244.5,533.0
 L247.9,535.4 L251.4,537.3 L254.8,538.7 L258.3,539.7 L261.7,540.2 L265.1,540.3 L268.6,539.9 L272.0,539.1
 L275.4,538.0 L278.9,536.4 L282.3,537.6 L285.8,535.2 L289.2,532.3 L292.6,529.1 L296.1,525.6 L299.5,521.9
 L303.0,518.1 L306.4,514.1 L309.8,513.2 L313.3,508.6 L316.7,504.4 L320.2,501.9 L323.6,497.7 L327.0,493.4
 L330.5,489.0 L333.9,484.8 L337.4,480.9 L340.8,477.2 L344.2,473.6 L347.7,470.4 L351.1,467.4 L354.6,464.7
 L358.0,462.3 L361.4,460.3 L364.9,458.5 L368.3,457.1 L371.8,455.9 L375.2,455.2 L378.6,454.7 L382.1,454.5
 L385.5,454.7 L389.0,455.1 L392.4,455.9 L395.8,456.9 L399.3,458.2 L402.7,459.7 L406.2,461.5 L409.6,463.5
 L413.0,465.7 L416.5,470.4 L419.9,473.1 L423.3,475.3 L426.8,477.6 L430.2,480.0 L433.7,482.6 L437.1,485.3
 L440.5,488.2 L444.0,491.2 L447.4,494.3 L450.9,497.4 L454.3,500.6 L457.7,503.7 L461.2,506.9 L464.6,512.7
 L468.1,515.8 L471.5,518.6 L474.9,521.1 L478.4,523.6 L481.8,526.0 L485.3,528.3 L488.7,530.6 L492.1,532.7
 L495.6,534.8 L499.0,536.8 L502.5,538.6 L505.9,540.4 L509.3,542.0 L512.8,543.5 L516.2,544.8 L519.7,546.8
 L523.1,547.9 L526.5,548.8 L530.0,549.5 L533.4,550.1 L536.9,550.6 L540.3,551.0 L543.7,551.3 L547.2,551.5
 L550.6,551.6 L554.0,551.6 L557.5,551.5 L560.9,551.3 L564.4,551.1 L567.8,550.8 L571.2,550.4 L574.7,549.9
 L578.1,549.4 L581.6,548.9 L585.0,548.3 L588.4,547.7 L591.9,547.0 L595.3,546.4 L598.8,545.7 L602.2,545.0
 L605.6,544.4 L609.1,543.7 L612.5,543.0 L616.0,542.4 L619.4,541.8 L622.8,541.2 L626.3,540.6 L629.7,540.0
 L633.2,539.5 L636.6,539.1 L640.0,538.7 L643.5,538.3 L646.9,538.0 L650.4,537.7 L653.8,537.5 L657.2,537.3
 L660.7,537.2 L664.1,537.1 L667.6,537.1 L671.0,537.2 L674.4,537.3 L677.9,537.4 L681.3,537.6 L684.7,537.9
 L688.2,538.2 L691.6,538.5 L695.1,538.9 L698.5,539.4 L701.9,539.9 L705.4,540.4 L708.8,540.9 L712.3,541.5
 L715.7,542.1 L719.1,542.8 L722.6,543.4 L726.0,544.1 L729.5,544.8 L732.9,545.6 L736.3,546.3 L739.8,547.0
 L743.2,547.8 L746.7,548.5 L750.1,549.2 '/> 
 
 m4Oh01eta04 surfaceEnergy J250-dtmin 
 
 
 m4 O h01 e ta04 surfaceEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M288.8,81.1 L342.2,81.1 M62.2,49.2 L63.2,49.1 L64.3,49.4 L65.3,49.6 L66.3,50.0 L67.4,50.4
 L68.4,51.0 L69.4,51.6 L70.5,52.4 L71.5,53.3 L72.5,54.2 L73.6,55.3 L74.6,56.4 L75.6,57.6
 L76.6,59.0 L77.7,60.4 L78.7,61.8 L79.7,63.4 L80.8,65.0 L81.8,66.8 L82.8,68.5 L83.9,70.4
 L84.9,72.3 L85.9,74.3 L87.0,76.4 L88.0,78.6 L89.0,80.8 L90.1,83.1 L91.1,85.4 L92.1,87.8
 L93.2,90.2 L94.2,92.8 L95.2,95.

In [ ]:
pltTabE4[1].XrangeMin = 0;
pltTabE4[1].XrangeMax = 1.0;
var gp = pltTabE4[1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta04 kineticEnergy J250 
 
 
 
 
 m4 O h01 e ta04 kineticEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,564.0 L65.6,562.9 L69.1,559.9 L72.5,555.2 L76.0,549.0 L79.4,541.5
 L82.8,532.9 L86.3,523.2 L89.7,512.6 L93.2,501.1 L96.6,488.9 L100.0,476.0 L103.5,462.5 L106.9,448.5
 L110.4,434.1 L113.8,419.3 L117.2,404.3 L120.7,389.0 L124.1,373.7 L127.6,358.4 L131.0,343.1 L134.4,327.9
 L137.9,312.8 L141.3,298.0 L144.7,283.6 L148.2,269.4 L151.6,255.7 L155.1,242.5 L158.5,229.9 L161.9,217.7
 L165.4,206.1 L168.8,195.4 L172.3,184.8 L175.7,175.4 L179.1,166.8 L182.6,159.0 L186.0,152.1 L189.5,146.1
 L192.9,141.1 L196.3,136.9 L199.8,133.8 L203.2,131.5 L206.7,130.1 L210.1,129.6 L213.5,130.0 L217.0,131.3
 L220.4,133.5 L223.9,136.6 L227.3,140.4 L230.7,145.0 L234.2,150.4 L237.6,156.7 L241.1,163.7 L244.5,171.5
 L247.9,180.2 L251.4,189.5 L254.8,199.4 L258.3,210.0 L261.7,221.1 L265.1,232.8 L268.6,244.9 L272.0,257.5
 L275.4,270.4 L278.9,283.7 L282.3,297.2 L285.8,310.8 L289.2,324.6 L292.6,338.5 L296.1,352.3 L299.5,366.0
 L303.0,379.5 L306.4,392.8 L309.8,405.9 L313.3,418.6 L316.7,430.9 L320.2,442.8 L323.6,454.1 L327.0,464.9
 L330.5,475.1 L333.9,484.8 L337.4,493.8 L340.8,502.1 L344.2,509.8 L347.7,516.9 L351.1,523.3 L354.6,529.0
 L358.0,534.1 L361.4,538.5 L364.9,542.2 L368.3,545.3 L371.8,547.9 L375.2,549.8 L378.6,551.2 L382.1,552.1
 L385.5,552.5 L389.0,552.4 L392.4,551.8 L395.8,550.9 L399.3,549.6 L402.7,548.0 L406.2,546.1 L409.6,543.9
 L413.0,541.5 L416.5,539.1 L419.9,536.2 L423.3,533.3 L426.8,530.3 L430.2,527.3 L433.7,524.3 L437.1,521.3
 L440.5,518.4 L444.0,515.4 L447.4,512.5 L450.9,509.7 L454.3,507.1 L457.7,504.5 L461.2,502.0 L464.6,499.7
 L468.1,497.5 L471.5,495.4 L474.9,493.6 L478.4,492.0 L481.8,490.5 L485.3,489.3 L488.7,488.3 L492.1,487.4
 L495.6,486.8 L499.0,486.3 L502.5,486.1 L505.9,486.0 L509.3,486.1 L512.8,486.5 L516.2,487.0 L519.7,487.7
 L523.1,488.5 L526.5,489.5 L530.0,490.6 L533.4,492.0 L536.9,493.5 L540.3,495.1 L543.7,496.8 L547.2,498.6
 L550.6,500.5 L554.0,502.5 L557.5,504.6 L560.9,506.7 L564.4,508.8 L567.8,511.1 L571.2,513.3 L574.7,515.6
 L578.1,517.8 L581.6,520.1 L585.0,522.3 L588.4,524.5 L591.9,526.7 L595.3,528.8 L598.8,530.9 L602.2,532.9
 L605.6,534.9 L609.1,536.7 L612.5,538.5 L616.0,540.3 L619.4,541.9 L622.8,543.4 L626.3,544.8 L629.7,546.1
 L633.2,547.4 L636.6,548.5 L640.0,549.5 L643.5,550.4 L646.9,551.2 L650.4,551.8 L653.8,552.4 L657.2,552.9
 L660.7,553.2 L664.1,553.5 L667.6,553.7 L671.0,553.7 L674.4,553.7 L677.9,553.6 L681.3,553.4 L684.7,553.1
 L688.2,552.8 L691.6,552.4 L695.1,551.9 L698.5,551.4 L701.9,550.8 L705.4,550.2 L708.8,549.5 L712.3,548.8
 L715.7,548.1 L719.1,547.3 L722.6,546.6 L726.0,545.8 L729.5,545.0 L732.9,544.2 L736.3,543.5 L739.8,542.7
 L743.2,542.0 L746.7,541.2 L750.1,540.5 '/> 
 
 m4Oh01eta04 kineticEnergy J250-dtmin 
 
 
 m4 O h01 e ta04 kineticEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M288.8,81.1 L342.2,81.1 M62.2,564.0 L63.2,563.9 L64.3,563.6 L65.3,563.1 L66.3,562.4 L67.4,561.6
 L68.4,560.6 L69.4,559.4 L70.5,558.2 L71.5,556.7 L72.5,555.1 L73.6,553.4 L74.6,551.6 L75.6,549.7
 L76.6,547.6 L77.7,545.4 L78.7,543.1 L79.7,540.7 L80.8,538.2 L81.8,535.6 L82.8,532.9 L83.9,530.2
 L84.9,527.3 L85.9,524.3 L87.0,521.3 L88.0,518.1 L89.0,514.9 L90.1,511.6 L91.1,508.3 L92.1,504.8
 L93.2,501.3 L94.2,497.7 L95.2,494.1 L96.3,490.4 L97.3,486.6 L98.3,482.8 L99.3,478.9 L100

In [ ]:
pltTabE4[2].XrangeMin = 0;
pltTabE4[2].XrangeMax = 1.0;
var gp = pltTabE4[2].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.15 
 
 
 
 
 3.2 
 
 
 
 
 3.25 
 
 
 
 
 3.3 
 
 
 
 
 3.35 
 
 
 
 
 3.4 
 
 
 
 
 3.45 
 
 
 
 
 3.5 
 
 
 
 
 3.55 
 
 
 
 
 3.6 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta04 totalEnergy J250 
 
 
 
 
 m4 O h01 e ta04 totalEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M311.0,57.1 L364.4,57.1 M62.2,59.1 L65.6,58.4 L69.1,58.1 L72.5,57.9 L76.0,58.0 L79.4,58.3
 L82.8,58.9 L86.3,59.8 L89.7,60.9 L93.2,62.3 L96.6,63.9 L100.0,65.8 L103.5,68.1 L106.9,70.6
 L110.4,73.3 L113.8,76.2 L117.2,79.5 L120.7,83.0 L124.1,86.7 L127.6,90.8 L131.0,95.1 L134.4,99.7
 L137.9,104.5 L141.3,109.5 L144.7,114.7 L148.2,120.3 L151.6,126.0 L155.1,131.9 L158.5,138.1 L161.9,145.4
 L165.4,152.1 L168.8,158.4 L172.3,165.9 L175.7,172.9 L179.1,180.1 L182.6,187.3 L186.0,194.7 L189.5,202.1
 L192.9,209.6 L196.3,217.2 L199.8,225.4 L203.2,233.0 L206.7,240.4 L210.1,248.1 L213.5,255.3 L217.0,262.6
 L220.4,270.0 L223.9,277.4 L227.3,284.9 L230.7,292.6 L234.2,300.2 L237.6,307.9 L241.1,317.2 L244.5,324.8
 L247.9,332.0 L251.4,339.0 L254.8,346.0 L258.3,352.9 L261.7,359.6 L265.1,366.1 L268.6,372.5 L272.0,378.7
 L275.4,384.7 L278.9,390.5 L282.3,399.2 L285.8,404.4 L289.2,409.2 L292.6,413.6 L296.1,417.8 L299.5,421.7
 L303.0,425.4 L306.4,428.9 L309.8,435.2 L313.3,437.7 L316.7,440.3 L320.2,444.4 L323.6,446.5 L327.0,448.2
 L330.5,449.5 L333.9,450.7 L337.4,451.7 L340.8,452.6 L344.2,453.4 L347.7,454.1 L351.1,454.7 L354.6,455.1
 L358.0,455.6 L361.4,455.9 L364.9,456.3 L368.3,456.6 L371.8,456.9 L375.2,457.1 L378.6,457.5 L382.1,457.8
 L385.5,458.1 L389.0,458.5 L392.4,459.0 L395.8,459.5 L399.3,460.0 L402.7,460.7 L406.2,461.4 L409.6,462.2
 L413.0,463.1 L416.5,466.4 L419.9,467.5 L423.3,468.1 L426.8,468.7 L430.2,469.5 L433.7,470.4 L437.1,471.5
 L440.5,472.7 L444.0,474.1 L447.4,475.6 L450.9,477.1 L454.3,478.8 L457.7,480.5 L461.2,482.3 L464.6,486.8
 L468.1,488.8 L471.5,490.3 L474.9,491.9 L478.4,493.5 L481.8,495.1 L485.3,496.7 L488.7,498.4 L492.1,500.1
 L495.6,501.8 L499.0,503.5 L502.5,505.2 L505.9,506.9 L509.3,508.6 L512.8,510.3 L516.2,511.9 L519.7,514.2
 L523.1,515.8 L526.5,517.3 L530.0,518.6 L533.4,520.0 L536.9,521.3 L540.3,522.6 L543.7,523.8 L547.2,525.0
 L550.6,526.2 L554.0,527.3 L557.5,528.3 L560.9,529.3 L564.4,530.3 L567.8,531.2 L571.2,532.1 L574.7,532.9
 L578.1,533.6 L581.6,534.3 L585.0,535.0 L588.4,535.6 L591.9,536.2 L595.3,536.7 L598.8,537.2 L602.2,537.6
 L605.6,538.0 L609.1,538.4 L612.5,538.7 L616.0,539.1 L619.4,539.3 L622.8,539.6 L626.3,539.8 L629.7,540.0
 L633.2,540.2 L636.6,540.3 L640.0,540.5 L643.5,540.6 L646.9,540.7 L650.4,540.8 L653.8,540.9 L657.2,541.0
 L660.7,541.1 L664.1,541.2 L667.6,541.2 L671.0,541.3 L674.4,541.4 L677.9,541.5 L681.3,541.6 L684.7,541.7
 L688.2,541.8 L691.6,542.0 L695.1,542.1 L698.5,542.2 L701.9,542.4 L705.4,542.6 L708.8,542.7 L712.3,542.9
 L715.7,543.2 L719.1,543.4 L722.6,543.6 L726.0,543.9 L729.5,544.2 L732.9,544.4 L736.3,544.7 L739.8,545.0
 L743.2,545.4 L746.7,545.7 L750.1,546.1 '/> 
 
 m4Oh01eta04 totalEnergy J250-dtmin 
 
 
 m4 O h01 e ta04 totalEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M311.0,81.1 L364.4,81.1 M62.2,59.1 L63.2,58.9 L64.3,59.0 L65.3,59.0 L66.3,58.9 L67.4,58.9
 L68.4,58.9 L69.4,59.0 L70.5,59.0 L71.5,59.1 L72.5,59.2 L73.6,59.3 L74.6,59.4 L75.6,59.5
 L76.6,59.7 L77.7,59.9 L78.7,60.1 L79.7,60.3 L80.8,60.6 L81.8,60.9 L82.8,61.2 L83.9,61.5
 L84.9,61.8 L85.9,62.2 L87.0,62.5 L88.0,62.9 L89.0,63.4 L90.1,63.8 L91.1,64.3 L92.1,64.8
 L93.2,65.3 L94.2,65.8 L95.2,66.4 L96.3,67.0 L97.

In [ ]:
Plot2Ddata[] pltTabE5 = PlotTableEnrgy[4];

In [ ]:
pltTabE5[0].XrangeMin = 0;
pltTabE5[0].XrangeMax = 0.5;
var gp = pltTabE5[0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta07 surfaceEnergy J250 
 
 
 
 
 m4 O h01 e ta07 surfaceEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M53.9,88.7 L60.9,89.5 L67.8,89.2 L74.8,90.3 L81.7,92.1 L88.7,94.2
 L95.7,96.3 L102.6,97.8 L109.6,98.7 L116.6,103.7 L123.5,109.8 L130.5,115.3 L137.4,121.6 L144.4,128.5
 L151.4,134.9 L158.3,141.0 L165.3,148.6 L172.3,156.5 L179.2,164.5 L186.2,172.8 L193.1,175.8 L200.1,184.4
 L207.1,193.1 L214.0,202.0 L221.0,210.9 L228.0,220.0 L234.9,229.1 L241.9,238.3 L248.8,247.4 L255.8,256.6
 L262.8,265.7 L269.7,274.8 L276.7,283.9 L283.6,295.2 L290.6,303.5 L297.6,312.0 L304.5,321.8 L311.5,329.6
 L318.5,338.5 L325.4,347.0 L332.4,355.0 L339.3,363.2 L346.3,369.5 L353.3,376.7 L360.2,384.2 L367.2,391.7
 L374.2,399.1 L381.1,406.3 L388.1,413.4 L395.0,420.6 L402.0,428.0 L409.0,439.0 L415.9,446.6 L422.9,453.7
 L429.8,459.4 L436.8,464.9 L443.8,470.1 L450.7,475.0 L457.7,479.6 L464.7,483.8 L471.6,487.7 L478.6,491.2
 L485.5,495.6 L492.5,498.3 L499.5,500.9 L506.4,503.3 L513.4,505.3 L520.4,507.2 L527.3,511.0 L534.3,512.8
 L541.2,514.1 L548.2,514.7 L555.2,515.4 L562.1,516.0 L569.1,516.5 L576.1,516.8 L583.0,517.0 L590.0,520.0
 L596.9,522.0 L603.9,519.6 L610.9,518.7 L617.8,517.6 L624.8,516.5 L631.7,515.9 L638.7,514.7 L645.7,514.2
 L652.6,514.8 L659.6,514.5 L666.6,512.7 L673.5,511.3 L680.5,509.8 L687.4,509.4 L694.4,507.8 L701.4,506.0
 L708.3,503.4 L715.3,501.1 L722.3,498.8 L729.2,498.2 L736.2,495.2 L743.1,492.6 L750.1,490.1 '/> 
 
 m4Oh01eta07 surfaceEnergy J250-dtmin 
 
 
 m4 O h01 e ta07 surfaceEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M288.8,81.1 L342.2,81.1 M53.9,88.7 L56.0,89.5 L58.1,88.8 L60.2,88.8 L62.3,89.0 L64.3,89.3
 L66.4,89.7 L68.5,90.0 L70.6,90.5 L72.7,91.0 L74.8,91.5 L76.9,92.1 L79.0,92.7 L81.1,93.3
 L83.1,94.0 L85.2,94.7 L87.3,95.4 L89.4,96.1 L91.5,96.7 L93.6,97.3 L95.7,97.7 L97.8,98.0
 L99.8,98.1 L101.9,98.2 L104.0,98.4 L106.1,99.0 L108.2,100.3 L110.3,102.0 L112.4,103.9 L114.5,105.9
 L116.6,106.2 L118.6,107.1 L120.7,109.2 L122.8,111.2 L124.9,113.3 L127.0,115.3 L129.1,117.2 L131.2,119.1
 L133.3,121.1 L135.4,123.0 L137.4,125.0 L139.5,126.9 L141.6,128.9 L143.7,129.8 L145.8,131.0 L147.9,133.3
 L150.0,135.6 L152.1,137.9 L154.2,140.2 L156.2,142.5 L158.3,144.8 L160.4,147.1 L162.5,149.5 L164.6,151.8
 L166.7,154.2 L168.8,156.6 L170.9,159.0 L173.0,161.5 L175.0,164.0 L177.1,166.5 L179.2,169.0 L181.3,171.5
 L183.4,173.9 L185.5,173.4 L187.6,175.8 L189.7,174.8 L191.7,177.3 L193.8,179.8 L195.9,182.4 L198.0,185.0
 L200.1,187.6 L202.2,190.2 L204.3,192.8 L206.4,195.5 L208.5,198.1 L210.5,200.8 L212.6,203.4 L214.7,206.1
 L216.8,208.8 L218.9,211.5 L221.0,214.2 L223.1,216.9 L225.2,219.6 L227.3,222.3 L229.3,225.0 L231.4,227.7
 L233.5,230.5 L235.6,233.2 L237.7,235.9 L239.8,238.6 L241.9,241.3 L244.0,244.1 L246.1,246.8 L248.1,249.5
 L250.2,252.2 L252.3,257.0 L254.4,259.2 L256.5,261.7 L258.6,264.3 L260.7,266.9 L262.8,269.5 L264.8,272.1
 L266.9,274.7 L269.0,277.3 L271.1,280.0 L273.2,282.6 L275.3,285.2 L277.4,287.8 L279.5,290.4 L281.6,293.0
 L283.6,298.5 L285.7,300.4 L287.8,302.7 L289.9,305.1 L292.0,307.5 L294.1,310.0 L296.2,312.4 L298.3,314.9
 L300.4,319.8 L302.4,322.3 L304.5,324.1 L306.6,326.3 L308.7,328.6 L310.8,331.0 L312.9,335.1 L315.0,337.6
 L317.1,339.9 L319.2,342.4 L321.2,345.0 L323.3,347.2 L325.4,348.2 L327.5,350.0 L329.6,352.2 L331.7,354.5
 L333.8,356.8 L335.9,359.2 L337.9,361.5 

In [ ]:
pltTabE5[1].XrangeMin = 0;
pltTabE5[1].XrangeMax = 0.5;
var gp = pltTabE5[1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0.3 
 
 
 
 
 0.35 
 
 
 
 
 0.4 
 
 
 
 
 0.45 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta07 kineticEnergy J250 
 
 
 
 
 m4 O h01 e ta07 kineticEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M288.8,57.1 L342.2,57.1 M62.2,564.0 L69.1,562.9 L76.0,560.2 L82.8,556.1 L89.7,550.4 L96.6,544.4
 L103.5,537.6 L110.4,529.9 L117.2,521.3 L124.1,512.0 L131.0,502.0 L137.9,491.3 L144.7,479.9 L151.6,468.1
 L158.5,455.6 L165.4,442.8 L172.3,429.4 L179.1,413.7 L186.0,401.3 L192.9,388.1 L199.8,374.5 L206.7,360.6
 L213.5,346.4 L220.4,332.8 L227.3,319.0 L234.2,305.3 L241.1,291.7 L247.9,278.2 L254.8,264.9 L261.7,251.8
 L268.6,239.1 L275.4,226.6 L282.3,214.6 L289.2,203.1 L296.1,191.7 L303.0,180.7 L309.8,170.2 L316.7,160.1
 L323.6,150.4 L330.5,141.1 L337.4,132.3 L344.2,116.6 L351.1,114.3 L358.0,108.4 L364.9,102.9 L371.8,97.5
 L378.6,92.6 L385.5,88.4 L392.4,84.6 L399.3,81.1 L406.2,78.2 L413.0,76.0 L419.9,74.2 L426.8,72.4
 L433.7,72.5 L440.5,73.5 L447.4,75.2 L454.3,77.6 L461.2,80.6 L468.1,84.3 L474.9,88.6 L481.8,93.5
 L488.7,98.8 L495.6,104.9 L502.5,111.2 L509.3,118.0 L516.2,125.1 L523.1,132.6 L530.0,140.2 L536.9,148.4
 L543.7,157.0 L550.6,165.9 L557.5,175.1 L564.4,184.5 L571.2,194.1 L578.1,203.9 L585.0,214.0 L591.9,224.0
 L598.8,234.5 L605.6,245.5 L612.5,256.5 L619.4,267.6 L626.3,278.9 L633.2,290.3 L640.0,301.8 L646.9,313.4
 L653.8,325.5 L660.7,337.4 L667.6,349.3 L674.4,361.0 L681.3,372.9 L688.2,385.0 L695.1,396.8 L701.9,408.5
 L708.8,419.9 L715.7,431.2 L722.6,442.1 L729.5,452.7 L736.3,462.9 L743.2,472.5 L750.1,481.7 '/> 
 
 m4Oh01eta07 kineticEnergy J250-dtmin 
 
 
 m4 O h01 e ta07 kineticEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M288.8,81.1 L342.2,81.1 M62.2,564.0 L64.3,563.8 L66.3,563.5 L68.4,563.0 L70.5,562.4 L72.5,561.6
 L74.6,560.7 L76.6,559.7 L78.7,558.6 L80.8,557.3 L82.8,556.0 L84.9,554.4 L87.0,552.3 L89.0,545.0
 L91.1,546.3 L93.2,546.2 L95.2,545.0 L97.3,543.4 L99.3,541.5 L101.4,539.5 L103.5,537.3 L105.5,535.1
 L107.6,532.8 L109.7,530.5 L111.7,528.0 L113.8,525.5 L115.9,522.9 L117.9,520.2 L120.0,517.4 L122.0,514.6
 L124.1,511.8 L126.2,508.8 L128.2,505.8 L130.3,502.8 L132.4,499.7 L134.4,496.5 L136.5,493.3 L138.6,490.0
 L140.6,486.6 L142.7,483.2 L144.7,479.8 L146.8,476.2 L148.9,470.7 L150.9,467.5 L153.0,464.7 L155.1,461.4
 L157.1,457.9 L159.2,454.3 L161.3,450.5 L163.3,446.7 L165.4,442.8 L167.4,438.9 L169.5,434.9 L171.6,430.8
 L173.6,426.4 L175.7,327.3 L177.8,394.4 L179.8,403.7 L181.9,404.4 L184.0,401.3 L186.0,376.8 L188.1,387.9
 L190.1,388.6 L192.2,386.5 L194.3,383.3 L196.3,379.8 L198.4,376.0 L200.5,372.1 L202.5,368.2 L204.6,364.1
 L206.7,360.1 L208.7,356.0 L210.8,351.9 L212.9,347.9 L214.9,343.9 L217.0,339.9 L219.0,335.9 L221.1,331.7
 L223.2,327.9 L225.2,323.9 L227.3,319.8 L229.4,315.8 L231.4,311.7 L233.5,307.7 L235.6,303.7 L237.6,299.6
 L239.7,295.6 L241.7,291.6 L243.8,287.6 L245.9,283.6 L247.9,279.6 L250.0,275.7 L252.1,271.7 L254.1,267.8
 L256.2,263.9 L258.3,260.4 L260.3,256.3 L262.4,252.4 L264.4,248.6 L266.5,244.8 L268.6,241.0 L270.6,237.3
 L272.7,233.6 L274.8,229.9 L276.8,226.1 L278.9,222.6 L281.0,219.2 L283.0,215.7 L285.1,212.2 L287.1,208.8
 L289.2,205.4 L291.3,202.0 L293.3,198.7 L295.4,195.4 L297.5,192.1 L299.5,188.8 L301.6,184.5 L303.7,181.6
 L305.7,178.8 L307.8,176.0 L309.8,173.0 L311.9,170.0 L314.0,167.0 L316.0,164.1 L318.1,161.0 L320.2,158.2
 L322.2,155.4 L324.3,152.5 L326.4,149.7 L328

In [ ]:
pltTabE5[2].XrangeMin = 0;
pltTabE5[2].XrangeMax = 0.5;
var gp = pltTabE5[2].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.3 
 
 
 
 
 3.4 
 
 
 
 
 3.5 
 
 
 
 
 3.6 
 
 
 
 
 3.7 
 
 
 
 
 3.8 
 
 
 
 
 3.9 
 
 
 
 
 4 
 
 
 
 
 4.1 
 
 
 
 
 4.2 
 
 
 
 
 4.3 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta07 totalEnergy J250 
 
 
 
 
 m4 O h01 e ta07 totalEnergy J250 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='3.0,6.0' d='M311.0,57.1 L364.4,57.1 M53.9,77.3 L60.9,77.8 L67.8,76.2 L74.8,75.7 L81.7,75.2 L88.7,75.0
 L95.7,74.5 L102.6,72.8 L109.6,70.1 L116.6,71.9 L123.5,74.7 L130.5,76.4 L137.4,78.9 L144.4,81.8
 L151.4,83.9 L158.3,85.5 L165.3,88.6 L172.3,90.9 L179.2,95.0 L186.2,99.1 L193.1,96.5 L200.1,100.5
 L207.1,104.6 L214.0,109.1 L221.0,113.7 L228.0,118.4 L234.9,123.2 L241.9,128.2 L248.8,133.1 L255.8,138.3
 L262.8,143.5 L269.7,148.8 L276.7,154.3 L283.6,162.7 L290.6,167.5 L297.6,172.8 L304.5,179.8 L311.5,184.6
 L318.5,190.9 L325.4,196.9 L332.4,202.6 L339.3,205.4 L346.3,211.9 L353.3,217.9 L360.2,224.4 L367.2,231.0
 L374.2,237.6 L381.1,244.3 L388.1,251.2 L395.0,258.2 L402.0,265.8 L409.0,278.0 L415.9,286.3 L422.9,294.1
 L429.8,300.9 L436.8,308.0 L443.8,315.0 L450.7,321.9 L457.7,328.8 L464.7,335.5 L471.6,342.1 L478.6,348.5
 L485.5,356.2 L492.5,362.2 L499.5,368.2 L506.4,374.0 L513.4,379.7 L520.4,385.3 L527.3,393.3 L534.3,399.2
 L541.2,404.6 L548.2,409.4 L555.2,414.3 L562.1,419.3 L569.1,424.1 L576.1,428.9 L583.0,433.7 L590.0,441.9
 L596.9,448.9 L603.9,451.0 L610.9,454.8 L617.8,458.6 L624.8,462.4 L631.7,466.7 L638.7,470.5 L645.7,475.1
 L652.6,481.3 L659.6,486.3 L666.6,489.5 L673.5,493.0 L680.5,496.5 L687.4,501.5 L694.4,504.9 L701.4,508.0
 L708.3,510.1 L715.3,512.3 L722.3,514.5 L729.2,518.6 L736.2,519.6 L743.1,520.8 L750.1,521.9 '/> 
 
 m4Oh01eta07 totalEnergy J250-dtmin 
 
 
 m4 O h01 e ta07 totalEnergy J250-dtmin 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='3.0,6.0' d='M311.0,81.1 L364.4,81.1 M53.9,77.3 L56.0,78.2 L58.1,77.1 L60.2,76.9 L62.3,76.9 L64.3,76.9
 L66.4,76.9 L68.5,76.9 L70.6,77.0 L72.7,77.0 L74.8,77.0 L76.9,77.0 L79.0,76.8 L81.1,74.3
 L83.1,75.7 L85.2,76.5 L87.3,76.8 L89.4,76.9 L91.5,76.8 L93.6,76.5 L95.7,76.1 L97.8,75.4
 L99.8,74.6 L101.9,73.6 L104.0,72.7 L106.1,72.3 L108.2,72.6 L110.3,73.4 L112.4,74.5 L114.5,75.6
 L116.6,74.8 L118.6,74.6 L120.7,75.6 L122.8,76.8 L124.9,77.8 L127.0,78.7 L129.1,79.6 L131.2,80.5
 L133.3,81.3 L135.4,82.1 L137.4,82.9 L139.5,83.6 L141.6,83.5 L143.7,83.2 L145.8,83.3 L147.9,84.6
 L150.0,85.9 L152.1,87.0 L154.2,88.0 L156.2,89.1 L158.3,90.1 L160.4,91.1 L162.5,92.1 L164.6,93.1
 L166.7,93.9 L168.8,52.2 L170.9,85.4 L173.0,92.5 L175.0,95.8 L177.1,97.4 L179.2,89.4 L181.3,97.4
 L183.4,100.6 L185.5,99.0 L187.6,100.5 L189.7,97.6 L191.7,99.0 L193.8,100.3 L195.9,101.6 L198.0,102.9
 L200.1,104.2 L202.2,105.4 L204.3,106.8 L206.4,108.1 L208.5,109.5 L210.5,110.9 L212.6,112.3 L214.7,113.6
 L216.8,115.1 L218.9,116.5 L221.0,117.9 L223.1,119.4 L225.2,120.8 L227.3,122.2 L229.3,123.7 L231.4,125.1
 L233.5,126.6 L235.6,128.1 L237.7,129.5 L239.8,131.0 L241.9,132.4 L244.0,134.0 L246.1,135.5 L248.1,137.0
 L250.2,138.5 L252.3,142.6 L254.4,143.4 L256.5,144.7 L258.6,146.0 L260.7,147.4 L262.8,148.8 L264.8,150.3
 L266.9,151.8 L269.0,153.3 L271.1,154.7 L273.2,156.3 L275.3,157.9 L277.4,159.5 L279.5,161.1 L281.6,162.6
 L283.6,167.6 L285.7,168.4 L287.8,169.7 L289.9,171.1 L292.0,172.5 L294.1,173.9 L296.2,174.9 L298.3,176.6
 L300.4,181.3 L302.4,183.0 L304.5,183.8 L306.6,185.0 L308.7,186.5 L310.8,188.0 L312.9,191.6 L315.0,193.3
 L317.1,194.8 L319.2,196.5 L321.2,198.4 L323.3,199.9 L325.4,199.9 L327.5,200.9 L329.6,202.4 L331.7,204.0
 